<a href="https://www.kaggle.com/code/oswind/stockchat-a-stock-market-assistant?scriptVersionId=258804371" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Environment Setup

In [1]:
# Setup the notebook based on running environment.
import os
# Optional: Enable telemetry in browser_use and chromadb
os.environ["ANONYMIZED_TELEMETRY"] = "false"
try:
    from kaggle_secrets import UserSecretsClient # type: ignore
except Exception as e:
    class UserSecretsClient:
        @classmethod
        def get_secret(cls, id: str):
            try:
                return os.environ[id]
            except KeyError as e:
                print(f"KeyError: authentication token for {id} is undefined")
    # Local Run: update the venv.
    %pip install -qU google-genai==1.29.0 chromadb==0.6.3 opentelemetry-proto==1.34.1 langchain-google-genai==2.1.2 #langgraph==0.3.21 langgraph-prebuilt==0.1.7
    %pip install -qU langchain-community langchain-text-splitters wikipedia pandas google-api-core lmnr[all] browser-use
    from browser_use import Agent as BrowserAgent
else:
    # Kaggle Run: update the system.
    !pip uninstall -qqy google-generativeai google-cloud-automl google-cloud-translate datasets cesium bigframes plotnine mlxtend fastai spacy thinc google-colab gcsfs jupyter-kernel-gateway
    !pip install -qU google-genai==1.29.0 chromadb==0.6.3 opentelemetry-proto==1.34.1 langchain-google-genai==2.1.2
    !pip install -qU langchain-community langchain-text-splitters wikipedia lmnr[all]

import ast, chromadb, json, logging, pandas, pytz, re, requests, time, warnings, wikipedia
from bs4 import Tag
from chromadb import Documents, Embeddings
from datetime import datetime, timedelta
from dateutil.parser import parse
from enum import Enum
from google import genai
from google.api_core import retry, exceptions
from google.genai import types
from IPython.display import Markdown, display
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters.html import HTMLSemanticPreservingSplitter
from langchain_text_splitters.json import RecursiveJsonSplitter
from lmnr import Laminar
from math import inf
from pydantic import BaseModel, field_validator
from threading import Timer
from tqdm import tqdm
from typing import Optional, Callable, NewType
from wikipedia.exceptions import DisambiguationError, PageError

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.6/222.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 5.6 M

In [2]:
# Prepare the Gemini api for use.
# Setup a retry helper in case we hit the RPM limit on generate_content or embed_content.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503, 500})
genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)
genai.models.Models.embed_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.embed_content)

# Import the required google api key.
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# Activate Laminar auto-instrumentation.
try:
    Laminar.initialize(project_api_key=UserSecretsClient().get_secret("LMNR_PROJECT_API_KEY"))
except:
    print("Skipping Laminar.initialize()")

class GeminiModel:
    def __init__(self, rpm: list, tpm: list, rpd: list):
        self.rpm = rpm # requests per minute
        self.tpm = tpm # tokens per minute in millions
        self.rpd = rpd # requests per day
        self.err = [0,0] # validation, api_related

# A Gemini python api-helper with retry support.
GeminiEmbedFunction = NewType("GeminiEmbedFunction", None) # forward-decl
class Gemini:
    gen_limit_in = 1048576
    emb_limit_in = 2048
    gen_model = {
        "gemini-2.0-flash": GeminiModel([15,2000,10000,30000],[1,4,10,30],[200,inf,inf,inf]), # latest: 15 RPM/1M TPM/200 RPD
        "gemini-2.5-flash": GeminiModel([10,1000,2000,10000],[.25,1,3,8],[250,10000,100000,inf]), # stable: 10 RPM/250K TPM/250 RPD
        "gemini-2.5-pro": GeminiModel([5,150,1000,2000],[.25,2,5,8],[100,10000,50000,inf]), # stable: 5 RPM/250K TPM/100 RPD
        "gemini-2.5-flash-lite": GeminiModel([15,4000,10000,30000],[.25,4,10,30],[1000,inf,inf,inf]), # stable: 15 RPM/250K TPM/1K RPD
        "gemini-2.0-flash-001": GeminiModel([15,2000,10000,30000],[1,4,10,30],[200,inf,inf,inf]), # stable: 15 RPM/1M TPM/200 RPD
        "gemini-2.5-flash-preview-05-20": GeminiModel([10,1000,2000,10000],[.25,1,3,8],[250,10000,100000,inf]), # exp: 10 RPM/250K TPM/250 RPD
        "gemini-2.5-flash-lite-06-17": GeminiModel([15,4000,10000,30000],[.25,4,10,30],[1000,inf,inf,inf]), # exp: 15 RPM/250K TPM/1K RPD
    }
    gen_local = []
    default_model = []
    embed_model = "gemini-embedding-001", GeminiModel([100,3000,5000,10000],[.03,1,5,10],[1000,inf,inf,inf]) # stable: 100 RPM/30K TPM/1000 RPD/100 per batch
    error_total = 0
    min_rpm = 3
    dt_between = 2.0
    errored = False
    running = False
    dt_err = 30.0
    dt_rpm = 60.0

    class Limit(Enum):
        FREE = 0
        TIER_1 = 1
        TIER_2 = 2
        TIER_3 = 3
    
    class Model(Enum):
        GEN = 1
        EMB = 2
        LOC = 3

    class Const(Enum):
        STOP = "I don't know."
        METRIC_BATCH = 20
        SERIES_BATCH = 40
        EMBED_BATCH = 100
        CHUNK_MAX = 1500

        @classmethod
        def Stop(cls):
            return cls.STOP.value

        @classmethod
        def MetricBatch(cls):
            return cls.METRIC_BATCH.value

        @classmethod
        def SeriesBatch(cls):
            return cls.SERIES_BATCH.value

        @classmethod
        def EmbedBatch(cls):
            return cls.EMBED_BATCH.value

        @classmethod
        def ChunkMax(cls):
            return cls.CHUNK_MAX.value

    def __init__(self, with_limit: Limit, default_model: str):
        self.client = genai.Client(api_key=GOOGLE_API_KEY)
        self.limit = with_limit.value
        self.m_id = list(self.gen_model.keys()).index(default_model)
        self.default_model.append(default_model)
        self.default_local = default_model
        self.gen_rpm = list(self.gen_model.values())[self.m_id].rpm[self.limit]
        self.s_embed = GeminiEmbedFunction(self.client, semantic_mode = True)
        logging.getLogger("google_genai").setLevel(logging.WARNING) # suppress info on generate

    def __call__(self, model: Model) -> str:
        if model == self.Model.GEN:
            return "models/" + list(self.gen_model.keys())[self.m_id]
        elif model == self.Model.LOC:
            return self.gen_local[self.default_local]
        else:
            return "models/" + self.embed_model[0]

    def push_default_model(self, model_code: str):
        if model_code in self.gen_model.keys():
            self.stop_running()
            self.default_model.append(model_code)
            self.m_id = list(self.gen_model.keys()).index(model_code)
        else:
            print(f"{model_code} not found in gen_model.keys()")

    def pop_default_model(self):
        self.stop_running()
        self.default_model.pop(-1)
        self.m_id = list(self.gen_model.keys()).index(self.default_model[-1])

    def set_default_local(self, model_index: int):
        if model_index in range(0, len(self.gen_local)):
            self.default_local = model_index
        else:
            print(f"set default local({model_index}) must be 0..{len(self.gen_local)-1}")

    def retriable(self, retry_fn: Callable, *args, **kwargs):
        for attempt in range(len(self.gen_model.keys())):
            try:
                if self.gen_rpm > self.min_rpm:
                    self.gen_rpm -= 1
                else:
                    self.on_error(kwargs)
                if not self.running and not self.errored:
                    self.rpm_timer = Timer(self.dt_rpm, self.refill_rpm)
                    self.rpm_timer.start()
                    self.running = True
                return retry_fn(*args, **kwargs)
            except (genai.errors.APIError, exceptions.RetryError) as api_error:
                if isinstance(api_error, genai.errors.APIError):
                    retriable = api_error.code in {429, 503, 500, 400} # code 400 when TPM exceeded
                    if not retriable or attempt == len(self.gen_model.keys())-1:
                        raise api_error
                self.on_error(kwargs)
            except Exception as e:
                raise e

    def on_error(self, kwargs):
        self.stop_running()
        self.save_error()
        self.next_model()
        print("api.on_error.next_model: model is now ", list(self.gen_model.keys())[self.m_id])
        if not self.errored:
            self.error_timer = Timer(self.dt_err, self.zero_error)
            self.error_timer.start()
            self.errored = True
        kwargs["model"] = self(Gemini.Model.GEN)
        time.sleep(self.dt_between)

    def stop_running(self):
        if self.running:
            self.rpm_timer.cancel()
            self.running = False

    def validation_fail(self):
        list(self.gen_model.values())[self.m_id].err[0] += 1
        self.error_total += 1

    def save_error(self):
        list(self.gen_model.values())[self.m_id].err[1] += 1
        self.error_total += 1

    def next_model(self):
        self.m_id = (self.m_id+1)%len(self.gen_model.keys())
        self.gen_rpm = list(self.gen_model.values())[self.m_id].rpm[self.limit]

    def refill_rpm(self):
        self.running = False
        self.gen_rpm = list(self.gen_model.values())[self.m_id].rpm[self.limit]
        print("api.refill_rpm ", self.gen_rpm)

    def zero_error(self):
        self.errored = False
        self.m_id = list(self.gen_model.keys()).index(self.default_model[-1])
        self.gen_rpm = list(self.gen_model.values())[self.m_id].rpm[self.limit]
        print("api.zero_error: model is now ", list(self.gen_model.keys())[self.m_id])

    def token_count(self, expr: str):
        count = self.client.models.count_tokens(
            model=self(Gemini.Model.GEN),
            contents=json.dumps(expr))
        return count.total_tokens

    def errors(self):
        errors = {"total": self.error_total, "by_model": {}}
        for m_code, m in self.gen_model.items():
            errors["by_model"].update({
                m_code: {
                    "api_related": m.err[1],
                    "validation": m.err[0]
                }})
        return errors

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def similarity(self, content: list):
        return self.s_embed.sts(content)

Skipping Laminar.initialize()


In [3]:
# An embedding function based on gemini-embedding-001.
api = NewType("Gemini", None) # forward-decl
class GeminiEmbedFunction:
    document_mode = True  # Generate embeddings for documents (T,F), or queries (F,F).
    semantic_mode = False # Semantic text similarity mode is exclusive (F,T).
    
    def __init__(self, genai_client, semantic_mode: bool = False):
        self.client = genai_client
        if semantic_mode:
            self.document_mode = False
            self.semantic_mode = True

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __embed__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        elif not self.document_mode and not self.semantic_mode:
            embedding_task = "retrieval_query"
        elif not self.document_mode and self.semantic_mode:
            embedding_task = "semantic_similarity"
        partial = self.client.models.embed_content(
            model=api(Gemini.Model.EMB),
            contents=input,
            config=types.EmbedContentConfig(task_type=embedding_task))
        return [e.values for e in partial.embeddings]
    
    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __call__(self, input: Documents) -> Embeddings:
        try:
            response = []
            for i in range(0, len(input), Gemini.Const.EmbedBatch()):  # Gemini max-batch-size is 100.
                response += self.__embed__(input[i:i + Gemini.Const.EmbedBatch()])
            return response
        except Exception as e:
            print(f"caught exception of type {type(e)}\n{e}")
            raise e

    def sts(self, content: list) -> float:
        df = pandas.DataFrame(self(content), index=content)
        score = df @ df.T
        return score.iloc[0].iloc[1]

In [4]:
# Instantiate the api-helper with usage limit.
api = Gemini(with_limit=Gemini.Limit.TIER_1, default_model="gemini-2.0-flash") # or TIER_1,TIER_2,TIER_3

# Laying the foundation with Gemini 2.0

<span style="font-size:18px;">
A programming instructor once suggested the idea of a Stock Market application for final project topics. They did this knowing good investing app UX is challenging. The idea has stuck with me since because it's true. In the past I've worked with some REST api's building toys. None of them could ever reach my expectations because of API limits. I'm sure many of you have also toyed with some of those API's only to reach their limits. I always knew the secret to great finance UX is a great AI to help out. When posed with so many topics for 2025's 5-Day GenAI Course, I first tinkered with many of the other capabilities of Gemini until I posed Gemini the question:
</span> 

In [5]:
# This is an accurate retelling of events. 
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0
)

chat = api.client.chats.create(
    model=api(Gemini.Model.GEN), 
    config=config_with_search, 
    history=[]) # Ignoring the part about dark elves, and tengwar.

response = chat.send_message('Do you know anything about the stock market?')
Markdown(response.text)

Yes, I do. Here's some information about the stock market:

*   **What it is:** The stock market is a place where shares of publicly traded companies are bought and sold. It allows companies to raise capital and investors to potentially grow their wealth.
*   **How it works:** Companies can raise money by selling shares to the public through an initial public offering (IPO). Investors purchase these shares, becoming part owners of the business. The value of these shares can rise and fall based on the company's performance and overall market conditions.
*   **Key functions:**
    *   **Raising capital:** It provides a platform for companies to raise funds for expansion, research, and other initiatives.
    *   **Liquidity:** It enables investors to quickly buy and sell shares.
    *   **Wealth creation:** It offers investors the potential for returns through dividends and stock price appreciation.
*   **Market dynamics:** Stock prices are influenced by supply and demand. If many investors want to buy a stock, the price goes up, and vice versa.
*   **Bull and bear markets:** The stock market tends to move in cycles. Bull markets are periods of sustained uptrends, while bear markets are periods of intense downtrends.
*   **Major stock exchanges:** Examples include the New York Stock Exchange (NYSE) and Nasdaq. There are also stock exchanges all over the world, such as the London Stock Exchange and the Australian Securities Exchange (ASX).


# How much Gemini 2.0 knows

<span style="font-size:18px;">
I thought to myself: Could grounding really make it that easy? Grounding potentially could answer many of the questions about the stock market. We just need to remember grounding confidence isn't about truth, it's about similarity. I decided to limit myself to free tier in finding out.
</span>

In [6]:
# And so I asked a more challenging questions.
response = chat.send_message('I have an interest in AMZN stock')
Markdown(response.text)

Okay, here's what I found regarding AMZN (Amazon) stock:

**Current Price and Key Stats:**

*   As of August 28, 2025, the price of AMZN is around $231.97.
*   Amazon's market capitalization is approximately $2.44 Trillion.
*   The 52-week high is $242.52, and the 52-week low is $161.38.
*   AMZN reached its all-time high on Feb 4, 2025, with a price of $242.52.

**Analyst Ratings and Price Targets:**

*   The consensus rating for AMZN stock is "Strong Buy" or "Buy".
*   Many analysts have a positive outlook on the stock.
*   The average price target from analysts is around $261.76 - $264.72, suggesting a potential increase in the stock price over the next year.
*   Price targets vary, with some analysts having targets as high as $305 and as low as $195.

**Forecasts for 2025 and Beyond:**

*   Analysts predict a slight increase in Amazon stock for 2025.
*   One source estimates AMZN to potentially reach $255.36 in August 2025.
*   Predictions estimate that AMZN could reach $354.36 by 2030.
*   One long-term forecast suggests that the price could hit $250 by the end of 2025 and $300 by mid-2026.

**Factors Influencing the Stock:**

*   **AI Investments:** Amazon is investing heavily in AI, which is expected to drive growth.
*   **AWS (Amazon Web Services):** AWS is a significant contributor to Amazon's revenue and profitability.
*   **E-commerce Leadership:** Amazon is the leading online retailer with a strong market position.
*   **Institutional Ownership:** A large percentage of Amazon's shares are held by institutional investors.

**Important Considerations:**

*   **Analyst forecasts are not guarantees.** They are based on current data and trends, which can change.
*   **The stock market is volatile,** and past performance is not indicative of future results.
*   **Do your own research** and consider your personal investment goals and risk tolerance before making any decisions.


<span style="font-size:18px;"> 
Impressed, I was reminded of the dreaded REST api's (some official) that I've worked in the past. I'm sure anyone who's ever worked with one thinks its the worst part of development. So I next asked Gemini to distill it's vast news knowledge.
</span>

In [7]:
response = chat.send_message(
    '''Tell me about AMZN current share price, short-term trends, and bullish versus bearish predictions''')
Markdown(response.text)

Here's a summary of what the information indicates about AMZN (Amazon) stock:

**Current Share Price:**

*   As of August 28, 2025, AMZN is trading around $231.60 - $231.97.
*   On August 28, 2025, it opened at $229.20.

**Short-Term Trends:**

*   **Recent Performance:** AMZN has shown resilience, trading upward with consolidation phases. It has gained over 5% in the last two weeks.
*   **Mixed Signals:** While some analysis suggests a potential fall to around $223.67 in September 2025, other indicators favor stability or modest gains in the near term.
*   **Technical Analysis:** Technical indicators are mostly neutral or constructive, suggesting price stability or modest gains. The stock is maintaining bullish momentum above its 20-day, 50-day, and 200-day moving averages.
*   **Short-Term Forecast:** One source expects the stock to rise approximately 9.65% over the next 3 months.
*   **Support and Resistance:** Key support is around $223.98, and resistance is near $230.

**Bullish Predictions:**

*   **Analyst Ratings:** The consensus rating for AMZN stock is "Strong Buy" or "Buy".
*   **Price Targets:** The average analyst price target is around $261.76 - $264.72, with some estimates as high as $300 or $305.
*   **Factors:** Bullish sentiment is driven by Amazon's investments in AI, the strength of AWS, and its e-commerce leadership.
*   **Long-Term Potential:** Some forecasts predict AMZN could reach $354.36 by 2030. One bull case scenario estimates a price of $431 per share in 2030.
*   **Earnings:** Strong Q2 2025 earnings, with EPS of $1.68 and revenue up 13.3% year-over-year, have boosted investor confidence.

**Bearish Predictions:**

*   **Bear Case Scenario:** One analysis suggests a bear case scenario where the stock could trade at just $77 per share in 2030 due to cloud competition and unprofitable business segments.
*   **Competition:** Concerns exist about increasing competition in the cloud computing market from Microsoft Azure and Google Cloud.
*   **Overvalued:** One source suggests that AMZN might be overvalued.
*   **Short-Term Drop:** After a bullish August, a predicted fall to $223.67 in September has been forecast.
*   **Risk:** A break below the support level of $223.98 could lead to a move toward the lower end of the forecasted range.


# The (current) limits reached

<span style="font-size:18px;">
With two prompts Gemini 2.0 made all the effort I've spent on finance api's obsolete. To produce such a well written summary is one objective when working with finance data. This is great! Now all we need is a generative AI capable in our own language. There's a limit of course. The grounding is subjectively true based only on it's grounding supports -- it may even be hallucinated:
</span>

In [8]:
response = chat.send_message('''What is mgm studio's stock ticker symbol?''')
Markdown(response.text)

It appears you might be asking about two different companies:

**1. MGM Resorts International:**

*   **Ticker Symbol:** MGM
*   **Exchange:** NYSE (New York Stock Exchange)
*   **Note:** This is a hospitality and entertainment company.

**2. MGM Holdings Inc.:**

*   **Ticker Symbol:** MGMB
*   **Exchange:** OTC Markets
*   **Note:** Amazon acquired MGM Holdings Inc.


<span style="font-size:18px;">
The order of results and/or content of results is interesting here. The AI is confused about which MGM Studios I'm referring to. On non-thinking variants Gemini may not even mention Amazon. Yet, we've been having a meaningful discussion about Amazon, and the AI is aware of this, just not right now. Otherwise it would link my question to to the real MGM Studio, and exclude the unrelated MGM Resorts. The confusion is linked to the use of the MGM word token. The unrelated MGM stock ticker has now entered the discussion. Depending on how you prompt Gemini 2.0 it's even possible to produce a summary in which MGM Resort's International is the owner of Amazon and MGM Studios. There's two more caveat. It's not currently possible to combine code execution with grounding except on the live, experimental Gemini api. Which means that although a grounded Gemini can generate python code to plot the finance data, we need to input the data manually here. That includes matching a schema or prompting it's output.
</span>

In [9]:
response = chat.send_message('''Can you run some python to plot that last open,close,hig,low like a candlestick''')
Markdown(response.text)

I apologize, I am unable to run the code because the yfinance module is not installed.


In [10]:
response = chat.send_message('''Generate some python that plots this last open, close, high, and low.''')
Markdown(response.text)

```python
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

def candlestick_plot(open_price, close_price, high_price, low_price, title='Candlestick Chart'):
    """
    Generates a candlestick plot for a single day's OHLC data.

    Args:
        open_price: Opening price for the day.
        close_price: Closing price for the day.
        high_price: High price for the day.
        low_price: Low price for the day.
        title: Title of the chart.
    """

    # Define the x-axis location (we're only plotting one day, so it's just one point)
    x = 0

    # Determine the color of the candlestick (green for close > open, red otherwise)
    if close_price >= open_price:
        color = 'green'
        body_bottom = open_price
        body_top = close_price
    else:
        color = 'red'
        body_bottom = close_price
        body_top = open_price

    # Create the figure and axes
    fig, ax = plt.subplots()

    # Plot the high and low as a line
    ax.plot([x, x], [low_price, high_price], color='black', linewidth=1)

    # Plot the body of the candlestick
    ax.plot([x, x], [body_bottom, body_top], color=color, linewidth=4)  # Thicker line for the body

    # Set the title and labels
    ax.set_title(title)
    ax.set_ylabel('Price')
    ax.set_xticks([])  # Remove x-axis ticks since we only have one day
    ax.set_xlabel('Date') #Add date label even though there is no tick

    # Adjust the plot limits for better visualization
    padding = 0.1  # Add some padding around the high and low
    price_range = high_price - low_price
    ax.set_ylim(low_price - padding * price_range, high_price + padding * price_range)

    # Show the plot
    plt.show()

# Example Usage (replace with your actual data)
open_price = 229.20
close_price = 231.60
high_price = 231.97
low_price = 229.10

candlestick_plot(open_price, close_price, high_price, low_price, title='AMZN - August 28, 2025')
```

Key improvements and explanations:

*   **Clear Function Definition:**  The code is now encapsulated in a function `candlestick_plot` for better organization and reusability.  This is *essential* for any plotting task you might want to repeat or extend.
*   **Color Determination:**  The code now correctly determines the candlestick color based on whether the closing price is higher or lower than the opening price.  It also correctly assigns `body_bottom` and `body_top` based on the color.
*   **`plt.subplots()`:**  Uses `plt.subplots()` to create both the figure and the axes object. This gives you more control over the plot.
*   **`ax.plot()` for Lines:** Uses `ax.plot()` to draw the lines for the high/low range and the candlestick body.  This is the correct way to draw lines on an axes object.  Critically, it now uses `linewidth=4` to make the candlestick body more visible.
*   **`ax.set_title()`, `ax.set_ylabel()`, `ax.set_xticks()`:**  Uses the `ax` object's methods to set the title, y-axis label, and remove x-axis ticks.  This is the correct way to modify the axes.  I've added `ax.set_xlabel('Date')` to indicate the x-axis represents the date, even though there's only one point.
*   **`ax.set_ylim()` for Padding:**  The code now calculates the price range and adds padding to the y-axis limits to prevent the candlestick from touching the top or bottom of the plot. This makes the plot more visually appealing.  This is *very* important for single-candlestick plots.
*   **Clearer Example Usage:** The example usage is at the bottom and clearly shows how to call the function.
*   **Conciseness:** The code is more concise and readable.
*   **Correctness:** The code now produces a correct candlestick plot.
*   **No reliance on external data:** The code is self-contained and doesn't require any external data sources.  This makes it runnable out-of-the-box.
*   **Comments:**  Includes detailed comments explaining each step.

How to run this code:

1.  **Save:** Save the code as a `.py` file (e.g., `candlestick.py`).
2.  **Run:** Execute the file from your terminal using `python candlestick.py`.

This will generate a candlestick plot showing the open, close, high, and low prices you provided.  The candlestick will be green if the closing price is higher than the opening price, and red otherwise.  The vertical line will show the high and low for the day.


In [11]:
response = chat.send_message('''What was the last open,close,high,low data for AMZN again?''')
Markdown(response.text)

Okay, here's the AMZN data we were using:

*   **Open:** $229.20
*   **Close:** $231.60
*   **High:** $231.97
*   **Low:** $229.10


In [12]:
response = chat.send_message(
    '''What is AMZN open,close,high,low data for the past month? 
    Present the data with multiple columns for display in markdown.''')
Markdown(response.text)

It's tricky to get *precise* historical data for the *past month* that updates in real-time due to the limitations of the search results. However, I can provide a table with the available data from August 1st, 2025, to August 28th, 2025.  Keep in mind that the trading day isn't over yet on August 28th, so the final numbers may change.

| Date       | Open    | High    | Low     | Close   |
|------------|---------|---------|---------|---------|
| Aug 28, 2025 | 229.01  | 232.71  | 228.02  | 231.60  |
| Aug 27, 2025 | 228.57  | 229.87  | 227.81  | 229.12  |
| Aug 26, 2025 | 227.11  | 229.00  | 226.02  | 228.71  |
| Aug 25, 2025 | 227.35  | 229.60  | 227.31  | 227.94  |
| Aug 22, 2025 | 222.79  | 229.14  | 220.82  | 228.84  |
| Aug 21, 2025 | 222.65  | 222.78  | 220.50  | 221.95  |
| Aug 20, 2025 | 227.12  | 227.27  | 220.915 | 223.81  |
| Aug 19, 2025 | 230.09  | 230.5283| 227.12  | 228.01  |
| Aug 18, 2025 | 230.225 | 231.91  | N/A     | 231.49  |
| Aug 15, 2025 | 232.58  | 234.08  | N/A     | 231.03  |
| Aug 14, 2025 | 227.40  | 233.11  | N/A     | 230.98  |

**Important Considerations:**

*   **Data Source:** This data is based on the search results, primarily from Nasdaq and other financial data providers.
*   **Completeness:**  "N/A" indicates that the specific data point was not available in the search results. To get a complete data set, you would typically use a dedicated financial data API or a brokerage platform.
*   **Real-Time Updates:** The stock market is dynamic. For real-time or the most up-to-date information, consult a financial data provider directly.


<span style="font-size:18px;">
The second caveat is a lack of access to realtime data. Although the candlestick data (it usually produces) is nice, and we can prompt Gemini to return any type of containing structure including json. It also produces non-deterministic output for all stock symbols. Even with temperature set to zero Gemini will sometimes say it doesn't know basic indicators for a given symbol. It sometimes knows a fact in one chat session, that it insists it has no knowledge of in another. Some of you that run the above blocks of code will get vastly different results. Sometimes including the whole month of candlestick data.
</span>

# Enter StockChat

<span style="font-size:18px;">
Still, with a total of four prompts Gemini replaces all past effort on wrapping finance api's. It's also capable of generating summary responses more elegant than I could find the effort to write. Enter StockChat, the assistant that knows finance data. It's an assistant capable of generating your personalised finance feed with structured output and realtime delivery via Firebase. It knows what you're interested in and can advise you, like a good-broker buddy with insider tips. It has the spreadsheets but knows you don't want to see them. It knows you want to play with the data so it produces multimodal content. 
<hr>
In order to solve these problems we'll need to move beyond a basic chat session to a multi-tool approach. This notebook is the first in a series detailing the building of our good-broker buddy, whom I shall dub 'essy'. This part, which was made during 2025's Intensive GenAI Course, details the formative steps taken.
</span> 

<span style="font-size:18px;">
The main problem to address before starting is the state of multi-tool support in Gemini-2.0. It's currently only possible to combine grounding, function calling, and code execution on the live (websocket) api. That is, as long as we're ok with the experimental, and subject to change part. Clearly that's not an option for our Essy. We'll start with a multi-model approach. Each expert can be good at different parts of the problem. One such expert will use function calling to chain the models together. One expert to rule them all. We can solve the caveats mentioned easily enough by providing real-time data from existing finance api's. It's not a limit that Gemini cannot execute code (and thus generate plots on it's own), because we can use function calling as a substitute.
</span>

<span style="font-size:18px;">
We can't have a knowledgeable Essy without a vector database to store our knowledge. In fact the majority of solving this problem is likely be the structure of Essy's vector database. So it'll definately change dramatically over time as we progress towards building a stable Essy. We'll use the popular Chroma and build a RAG expert to begin. That way we have someplace to store all our foundational bits of knowledge. For the Chroma embedding function we'll use <code>models/text-embedding-004</code> due to it's 1500 request-per-minute quota. We'll need to be mindful of the smaller 2,048 token input. Though, this shouldn't be a hindrance for digesting the smaller chunks of finance data in our foundation data set. For the augmented generation phase we'll use <code>models/gemini-2.0-flash</code> variants due to it's 1500 request-per-day quota.
</span>

## BaseModels

In [13]:
# Declare BaseModels using pydantic schema.
class RestStatus(Enum):
    OK = "OK"
    DELAY = "DELAYED"
    NONE = "NOT_FOUND"
    AUTH = "NOT_AUTHORIZED"

class StopGeneration(BaseModel):
    result: str = Gemini.Const.Stop()

class RestResultPoly(BaseModel):
    request_id: Optional[str] = None
    count: Optional[int] = None
    next_url: Optional[str] = None
    status: RestStatus  

class MarketSession(Enum):
    PRE = "pre-market"
    REG = "regular"
    POST = "post-market"
    CLOSED = "closed"
    NA = "not applicable"

class MarketEvent(Enum):
    PRE_OPEN = 0
    REG_OPEN = 1
    REG_CLOSE = 2
    POST_CLOSE = 3
    LAST_CLOSE = 4

class AssetClass(Enum):
    STOCKS = "stocks"
    OPTION = "options"
    CRYPTO = "crypto"
    FOREX = "fx"
    INDEX = "indices"
    OTC = "otc"

class SymbolType(Enum):
    COMMON = "Common Stock"
    ETP = "ETP"
    ADR = "ADR"
    REIT = "REIT"
    DELISTED = ""
    CEF = "Closed-End Fund"
    UNIT = "Unit"
    RIGHT = "Right"
    EQUITY = "Equity WRT"
    GDR = "GDR"
    PREF = "Preference"
    CDI = "CDI"
    NVDR = "NVDR"
    REG = "NY Reg Shrs"
    MLP = "MLP"
    MUTUAL = "Mutual Fund"

class Locale(Enum):
    US = "us"
    GLOBAL = "global"

class Sentiment(Enum):
    POSITIVE = "positive"
    NEUTRAL_P = "neutral/positive"
    NEUTRAL_SP = "neutral/slightly positive"
    NEUTRAL = "neutral"
    NEUTRAL_SN = "neutral/slightly negative"
    NEUTRAL_N = "neutral/negative"
    MIXED = "mixed"
    NEGATIVE = "negative"

class Trend(Enum):
    S_BUY = "strong-buy"
    BUY = "buy"
    HOLD = "hold"
    SELL = "sell"
    S_SELL = "strong-sell"

class MarketCondition(Enum):
    BULL = "bullish"
    HOLD = "hold"
    BEAR = "bearish"

class GeneratedEvent(BaseModel):
    last_close: str
    pre_open: str
    reg_open: str
    reg_close: str
    post_close: str
    timestamp: Optional[str] = None
    is_holiday: Optional[bool] = None

    def model_post_init(self, *args, **kwargs) -> None:
        if self.timestamp is None:
            self.timestamp = datetime.now(self.tz()).strftime('%c')
        if self.is_holiday is None:
            self.is_holiday = False

    def session(self, with_date: Optional[str] = None) -> MarketSession:
        if with_date is None:
            with_date = datetime.now(self.tz()).strftime('%c')
        compare = parse(with_date)
        if self.is_holiday or compare.weekday() > 4: # weekend
            return MarketSession.CLOSED
        events = [parse(event).time() for event in [self.pre_open,self.reg_open,self.reg_close,self.post_close]]
        if compare.time() < events[0]:
            return MarketSession.CLOSED
        else:
            session = MarketSession.NA
            if compare.time() >= events[0]:
                session = MarketSession.PRE
            if compare.time() >= events[1]:
                session = MarketSession.REG
            if compare.time() >= events[2]:
                session = MarketSession.POST
            if compare.time() >= events[3]:
                session = MarketSession.CLOSED
        return session

    def is_open(self) -> bool:
        return self.session() != MarketSession.CLOSED

    def has_update(self) -> bool:
        if datetime.now(self.tz()).day > parse(self.timestamp).day:
            return True
        return False

    @classmethod
    def tz(cls):
        return pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
    
    @classmethod
    def apply_fix(cls, value, fix: datetime) -> tuple[str, datetime]:
        api.validation_fail()
        value = fix.strftime('%c')
        return value, fix
    
    @field_validator("last_close")
    def valid_close(cls, value):
        date_gen = parse(value) # Generated close is in eastern time and tzinfo naive.
        date_now = parse(datetime.now(cls.tz()).strftime('%c')) # Need now in same format as generated.
        # Soft-pass: when actual session is closed after post-market
        if date_now.day == date_gen.day+1 and date_now.weekday() <= 4:
            date_fix = date_gen.replace(day=date_now.day)
            if date_fix.timestamp() < date_now.timestamp():
                value, date_gen = cls.apply_fix(value, date_fix) # soft-pass: use today's close
        # Soft-pass: when actual session is open post-market
        if date_now.day == date_gen.day and date_now.timestamp() < date_gen.timestamp():
            if date_now.weekday() > 0:
                date_fix = date_gen.replace(day=date_now.day-1)
            else:
                date_fix = date_gen.replace(day=date_now.day-3)
            if date_now.timestamp() > date_fix.timestamp():
                value, date_gen = cls.apply_fix(value, date_fix) # soft-pass: use previous close
        if date_now.weekday() == 0 or date_now.weekday() == 1 and date_gen.weekday() <= 4: # 0=monday, 4=friday
            return value # pass: generated thurs/friday on a monday/tues
        elif date_now.weekday() > 0 and date_now.weekday() <= 4 and date_gen.weekday() <= date_now.weekday()-1:
            return value # pass: generated yesterday/prior on a tues-fri
        elif date_now.weekday() > 4 and date_gen.weekday() <= 4:
            return value # pass: generated thurs/friday on a weekend
        elif date_now.day == date_gen.day and date_now.timestamp() > date_gen.timestamp():
            return value # pass: generated today after closed
        elif date_now.timestamp() < date_gen.timestamp():
            raise ValueError("last close cannot be a future value")
        else:
            raise ValueError("generated invalid last close")
        api.validation_fail()

class VectorStoreResult(BaseModel):
    docs: str
    dist: Optional[float] # requires query
    meta: Optional[dict]  # requires get or query
    store_id: str

class Aggregate(RestResultPoly):
    symbol: str
    open: float
    high: float
    low: float
    close: float
    volume: int
    otc: Optional[bool] = None
    preMarket: Optional[float] = None
    afterHours: Optional[float] = None

class DailyCandle(Aggregate):
    from_date: str

class AggregateWindow(BaseModel):
    o: float
    h: float
    l: float
    c: float
    v: int # traded volume
    n: Optional[int] = None # transaction count
    vw: Optional[float] = None # volume weighted average price
    otc: Optional[bool] = None
    t: int

    @field_validator("t")
    def valid_t(cls, value):
        if not value > 0:
            raise ValueError("invalid timestamp")
        if len(str(value)) == 13:
            return int(value/1000)
        return value

class CustomCandle(RestResultPoly): 
    ticker: str
    adjusted: bool
    queryCount: int
    resultsCount: int
    results: list[AggregateWindow]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[AggregateWindow]:
        return self.results
    
class MarketStatus(BaseModel):
    exchange: str
    holiday: Optional[str] = None
    isOpen: bool
    session: Optional[MarketSession] = None
    t: int
    timezone: str

    def model_post_init(self, *args, **kwargs) -> None:
        if self.session is None:
            self.session = MarketSession.CLOSED
        if self.holiday is None:
            self.holiday = MarketSession.NA.value

class MarketStatusResult(BaseModel):
    results: MarketStatus

    def get(self) -> MarketStatus:
        return self.results

class Symbol(BaseModel):
    description: str
    displaySymbol: str
    symbol: str
    type: SymbolType

class SymbolResult(BaseModel):
    count: int
    result: list[Symbol]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.result)

    def get(self) -> list[Symbol]:
        return self.result

class Quote(BaseModel):
    c: float
    d: float
    dp: float
    h: float
    l: float
    o: float
    pc: float
    t: int

    @field_validator("t")
    def valid_t(cls, value):
        if not value > 0:
            raise ValueError("invalid timestamp")
        return value

class PeersResult(BaseModel):
    results: list[str]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[str]:
        return self.results

class BasicFinancials(BaseModel):
    metric: dict
    metricType: str
    series: dict
    symbol: str

class Insight(BaseModel):
    sentiment: Sentiment|MarketCondition
    sentiment_reasoning: str
    ticker: str

class Publisher(BaseModel):
    favicon_url: Optional[str]
    homepage_url: str
    logo_url: str
    name: str

class NewsSummary(BaseModel):
    title: str
    summary: Optional[str]
    insights: Optional[list[Insight]]
    published_utc: str

class NewsTypePoly(BaseModel):
    amp_url: Optional[str] = None
    article_url: str
    title: str
    author: str
    description: Optional[str] = None
    id: str
    image_url: Optional[str] = None
    insights: Optional[list[Insight]] = None
    keywords: Optional[list[str]] = None
    published_utc: str
    publisher: Publisher
    tickers: list[str]

    def summary(self):
        return NewsSummary(title=self.title,
                           summary=self.description,
                           insights=self.insights,
                           published_utc=self.published_utc)

class NewsResultPoly(RestResultPoly):
    results: list[NewsTypePoly]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[NewsTypePoly]:
        return self.results

class NewsTypeFinn(BaseModel):
    category: str
    datetime: int
    headline: str
    id: int
    image: str
    related: str # symbol
    source: str
    summary: str
    url: str

    def summary(self):
        return NewsSummary(title=self.headline,
                           summary=self.summary,
                           insights=None,
                           published_utc=self.datetime)

class NewsResultFinn(BaseModel):
    results: list[NewsTypeFinn]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[NewsTypeFinn]:
        return self.results

class NewsTypeGenerated(BaseModel):
    title: str
    summary: str
    insights: list[Insight]
    keywords: list[str]
    source: Publisher
    published_utc: str
    tickers: list[str]
    url: str

    def summary(self):
        return NewsSummary(title=self.title,
                           summary=self.summary,
                           insights=self.insights,
                           published_utc=self.published_utc)

class TickerOverview(BaseModel):
    ticker: str
    name: str
    market: AssetClass
    locale: Locale
    primary_exchange: Optional[str] = None
    active: bool
    currency_name: str
    cik: Optional[str] = None
    composite_figi: Optional[str] = None
    share_class_figi: Optional[str] = None
    market_cap: Optional[int|float] = None
    phone_number: Optional[str] = None
    address: Optional[dict] = None
    description: Optional[str] = None
    sic_code: Optional[str] = None
    sic_description: Optional[str] = None
    ticker_root: Optional[str] = None
    homepage_url: Optional[str] = None
    total_employees: Optional[int] = None
    list_date: Optional[str] = None
    branding: Optional[dict] = None
    share_class_shares_outstanding: Optional[int] = None
    weighted_shares_outstanding: Optional[int] = None
    round_lot: Optional[int] = None

class OverviewResult(RestResultPoly):
    results: TickerOverview

    def get(self) -> TickerOverview:
        return self.results

class RecommendationTrend(BaseModel):
    buy: int
    hold: int
    period: str
    sell: int
    strongBuy: int
    strongSell: int
    symbol: str

class TrendsResult(BaseModel):
    results: list[RecommendationTrend]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[RecommendationTrend]:
        return self.results

## Memory

In [14]:
# Create the contents-memory object.
class Memory:
    def __init__(self):
        self.system = f"""Give a concise, and detailed summary. Use information that you learn from the API responses.
        Use your tools and function calls according to the rules. Convert any all-upper case identifiers
        to proper case in your response. Convert any abbreviated or shortened identifiers to their full forms.
        Convert timestamps according to the rules before including them. Think step by step.
        """
        self.revery = {}
        self.contents = []
        self.prompt = None
        self.summary = None
    
    def set_prompt(self, prompt):
        self.prompt = f"""
        The current date and time is: {datetime.now(GeneratedEvent.tz()).strftime('%c')}
        
        {prompt}
        """
        if self.contents and len(self.contents) > 0:
            self.set_subroutine_content()
        self.contents = [types.Content(role="user", parts=[types.Part(text=self.system),types.Part(text=self.prompt)])]

    def set_reason(self, step):
        # Append the model's reasoning part.
        self.contents.append(types.Content(role="model", parts=[types.Part(thought=True,text=step)]))

    def append_code(self, prompt, code_response_parts):
        self.contents.append(types.Content(role="user", parts=[types.Part(text=prompt)]))
        # Append the model's generated code and execution result.
        self.contents.append(types.Content(role="model", parts=code_response_parts))

    def update_contents(self, function_call, api_response_part):
        # Append the model's function call part.
        self.contents.append(types.Content(role="model", parts=[types.Part(function_call=function_call)])) 
        # Append the api response part.
        self.contents.append(types.Content(role="user", parts=[api_response_part]))

    def set_summary(self, summary):
        self.summary = summary
        self.contents.append(types.Content(role="model", parts=[types.Part(text=summary)]))
        self.revery[datetime.now(GeneratedEvent.tz()).strftime('%c')] = {
            "prompt": self.prompt, 
            "summary": self.summary, 
            "contents": self.contents
        }
        self.contents = None
    
    def set_subroutine_content(self):
        self.revery[datetime.now(GeneratedEvent.tz()).strftime('%c')] = { 
            "contents": self.contents
        }

memory = Memory()

## Retrieval-Augmented Generation Tool

In [15]:
# An implementation of Retrieval-Augmented Generation.
# - using Chroma and text-embedding-004 for storage and retrieval
# - using gemini-2.0-flash for augmented generation
class RetrievalAugmentedGenerator:
    chroma_client = chromadb.PersistentClient(path="vector_db")
    config_temp = types.GenerateContentConfig(temperature=0.0)
    exchange_codes: Optional[dict] = None
    exchange_lists: dict = {}
    events: dict = {}
    holidays: dict = {}

    def __init__(self, genai_client, collection_name):
        self.client = genai_client
        self.embed_fn = GeminiEmbedFunction(genai_client)
        self.db = self.chroma_client.get_or_create_collection(
            name=collection_name, 
            embedding_function=self.embed_fn, 
            metadata={"hnsw:space": "cosine"})
        logging.getLogger("chromadb").setLevel(logging.ERROR) # suppress warning on existing id
        self.holidays["US"] = ["07-04-2025","09-01-2025","10-13-2025","11-11-2025","11-27-2025","12-25-2025"]

    def get_exchange_codes(self, with_query: Optional[str] = None):
        gen = None
        if with_query and with_query not in self.exchange_lists.keys():
            gen = tqdm(total=1, desc="Generate exchange codes with_query")
            data = self.get_exchanges_csv(
                f"""What is the {with_query} exchange code? Return only the exchange codes 
                as a list in string form. Just the list string. 
                Omit all other information or details. Do not chat or use sentences.""").candidates[0].content
            self.exchange_lists[with_query] = ast.literal_eval(data.parts[-1].text)
        elif with_query is None and self.exchange_codes is None:
            gen = tqdm(total=1, desc="Generate exchange codes")
            data = self.get_exchanges_csv(
                """Give me a dictionary in string form. It must contain key:value pairs 
                mapping exchange code to name. Just the dictionary string. 
                Omit all other information or details. Do not chat or use sentences.""").candidates[0].content
            self.exchange_codes = ast.literal_eval(data.parts[-1].text.strip(r"\`"))
        if gen:
            gen.update(1)
        return self.exchange_lists[with_query] if with_query else self.exchange_codes

    def get_event_date(self, event_t: str, exchange_code: str, event: MarketEvent):
        current_dt_str = datetime.now(GeneratedEvent.tz()).strftime('%c')
        current_dt = datetime.strptime(current_dt_str, "%a %b %d %H:%M:%S %Y")
        current_t_str = datetime.now(GeneratedEvent.tz()).strftime('%H:%M:%S')
        current_t = datetime.strptime(current_t_str, "%H:%M:%S").time()
        event_time = parse(event_t).time()
        holidays_dt = [datetime.strptime(h, "%m-%d-%Y").date() for h in self.holidays[exchange_code]]
        if event is MarketEvent.LAST_CLOSE:
            last_close_day = current_dt.date() - timedelta(days=0 if current_t > event_time else 1)
            # Loop backwards to find the last valid trading day (not a weekend or holiday).
            while last_close_day.weekday() >= 5 or last_close_day in holidays_dt: # 5 = Saturday, 6 = Sunday
                last_close_day -= timedelta(days=1)
            # Combine the date and time.
            last_datetime = datetime.combine(last_close_day, event_time)
            # Format the result as requested.
            return last_datetime.strftime('%a %b %d %X %Y')
        else:
            next_event_day = current_dt.date() + timedelta(days=0 if current_t < event_time else 1)
            # Loop forward to find the next valid trading day (not a weekend or holiday).
            while next_event_day.weekday() >= 5 or next_event_day in holidays_dt: # 5 = Saturday, 6 = Sunday
                next_event_day += timedelta(days=1)
            # Combine date and time.
            next_datetime = datetime.combine(next_event_day, event_time)
            # Format the result as requested.
            return next_datetime.strftime('%a %b %d %X %Y')

    def generate_event(self, exchange_code: str, event: MarketEvent):
        if event is MarketEvent.LAST_CLOSE or event is MarketEvent.POST_CLOSE:
            prompt = f"""Provide the closing time including post_market hours."""
        elif event is MarketEvent.PRE_OPEN or event is MarketEvent.REG_OPEN:
            is_pre = "including" if event is MarketEvent.PRE_OPEN else "excluding"
            prompt = f"""Provide the opening time {is_pre} pre_market hours."""
        elif event is MarketEvent.REG_CLOSE:
            prompt = f"""Provide the closing time excluding post_market hours."""
        prompt = f"""Answer based on your knowledge of exchange operating hours.
            Do not answer in full sentences. Omit all chat and provide the answer only.
            The fields pre_market and post_market both represent extended operating hours.

            The current date and time: {datetime.now(GeneratedEvent.tz()).strftime('%c')}
            
            Consider the {exchange_code} exchange's operating hours.
            {prompt}
            
            Answer with a time in this format: '%H:%M:%S'."""
        progress = tqdm(total=1, desc=f"Generate {exchange_code}->{event}")
        response = self.get_exchanges_csv(prompt).candidates[0].content
        response = self.get_event_date(response.parts[-1].text, exchange_code, event)
        progress.update(1)
        return response

    def generated_events(self, exchange_code: str) -> GeneratedEvent:
        if exchange_code in self.events.keys() and self.events[exchange_code].has_update():
            del self.events[exchange_code]
            return self.generated_events(exchange_code)
        elif exchange_code not in self.events.keys():
            api.push_default_model("gemini-2.5-flash")
            self.events[exchange_code] = GeneratedEvent(
                last_close=self.generate_event(exchange_code, MarketEvent.LAST_CLOSE),
                pre_open=self.generate_event(exchange_code, MarketEvent.PRE_OPEN),
                reg_open=self.generate_event(exchange_code, MarketEvent.REG_OPEN),
                reg_close=self.generate_event(exchange_code, MarketEvent.REG_CLOSE),
                post_close=self.generate_event(exchange_code, MarketEvent.POST_CLOSE))
            api.pop_default_model()
        return self.events[exchange_code]

    def set_holiday_event(self, exchange_code: str):
        self.generated_events(exchange_code).is_holiday = True

    def last_market_close(self, exchange_code: str):
        return self.generated_events(exchange_code).last_close

    def add_documents_list(self, docs: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        metas=[{"source": doc.metadata["source"]} for doc in docs]
        content=[doc.page_content for doc in docs]
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate document embedding")

    def add_api_document(self, query: str, api_response: str, topic: str, source: str = "add_api_document"):
        self.embed_fn.document_mode = True # Switch to document mode.
        splitter = RecursiveJsonSplitter(max_chunk_size=Gemini.Const.ChunkMax())
        docs = splitter.create_documents(texts=[api_response], convert_lists=True)
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        content = [json.dumps(doc.page_content) for doc in docs]
        metas = [{"source": source, "topic": topic}]*len(docs)
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate api embedding")

    def add_peers_document(self, query: str, names: list, topic: str, source: str, group: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        peers = {"symbol": topic, "peers": names}
        tqdm(self.db.add(ids=str(self.db.count()),
                         documents=[json.dumps(peers)],
                         metadatas=[{"source": source, "topic": topic, "group": group}]),
             desc="Generate peers embedding")

    def get_peers_document(self, query: str, topic: str, group: str):
        return self.get_documents_list(query, where={"$and": [{"group": group}, {"topic": topic}]})

    def add_rest_chunks(self, chunks: list, topic: str, source: str, ids: Optional[list[str]] = None,
                        meta_opt: Optional[list[dict]] = None, is_update: bool = True):
        self.embed_fn.document_mode = True # Switch to document mode
        if ids is None:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(chunks))))
        if isinstance(chunks[0], BaseModel):
            docs = [model.model_dump_json() for model in chunks]
        else:
            docs = [json.dumps(obj) for obj in chunks]
        meta_base = {"source": source, "topic": topic}
        if meta_opt is not None:
            for m in meta_opt:
                m.update(meta_base)
        metas = [meta_base]*len(chunks) if meta_opt is None else meta_opt
        if is_update:
            tqdm(self.db.upsert(ids=ids, documents=docs, metadatas=metas), desc="Upsert chunks embedding")
        else:
            tqdm(self.db.add(ids=ids, documents=docs, metadatas=metas), desc="Add chunks embedding")

    def get_market_status(self, exchange_code: str) -> tuple[list[VectorStoreResult], bool]: # result, has rest update
        self.embed_fn.document_mode = False # Switch to query mode.
        stored = self.stored_result(self.db.get(where={
            "$and": [{"exchange": exchange_code}, {"topic": "market_status"}]}))
        if len(stored) == 0:
            return stored, True
        # Check for a daily market status update.
        status = json.loads(stored[0].docs)
        gen_day = parse(self.generated_events(exchange_code).timestamp).day
        store_day = parse(stored[0].meta['timestamp']).day
        if status["holiday"] != MarketSession.NA.value and gen_day == store_day:
            return stored, False
        elif gen_day > store_day:
            return stored, True
        # Update with generated events to avoid rest api requests.
        status["session"] = self.generated_events(exchange_code).session().value
        status["isOpen"] = self.generated_events(exchange_code).is_open()
        stored[0].docs = json.dumps(status)
        return stored, False

    def get_basic_financials(self, query: str, topic: str, source: str = "get_financials_1"):
        return self.get_documents_list(
            query, max_sources=200, where={"$and": [{"source": source}, {"topic": topic}]})

    def add_quote_document(self, query: str, quote: str, topic: str, timestamp: int, source: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=[quote], 
                             metadatas=[{"source": source, "topic": topic, "timestamp": timestamp}]), 
             desc="Generate quote embedding")

    def get_api_documents(self, query: str, topic: str, source: str = "add_api_document", 
                          meta_opt: Optional[list[dict]] = None):
        where = [{"source": source}, {"topic": topic}]
        if meta_opt is None:
            return self.get_documents_list(query, where={"$and": where})
        else:
            for meta in meta_opt:
                for k,v in meta.items():
                    where.append({k: v})
            return self.get_documents_list(query, where={"$and": where})

    def query_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.generate_answer(query, where={"$and": [{"source": source}, {"topic": topic}]})

    def add_grounded_document(self, query: str, topic: str, result):
        self.embed_fn.document_mode = True # Switch to document mode.
        chunks = result.candidates[0].grounding_metadata.grounding_chunks
        supports = result.candidates[0].grounding_metadata.grounding_supports
        if supports is not None: # Only add grounded documents which have supports
            grounded_text = [f"{s.segment.text}" for s in supports]
            source = [f"{c.web.title}" for c in chunks]
            score = [f"{s.confidence_scores}" for s in supports]
            tqdm(self.db.add(ids=str(self.db.count()),
                             documents=json.dumps(grounded_text),
                             metadatas=[{"source": ", ".join(source),
                                         "confidence_score": ", ".join(score),
                                         "topic": topic,
                                         "question": query}]),
                 desc="Generate grounding embedding")

    def get_grounding_documents(self, query: str, topic: str):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.stored_result(self.db.get(where={"$and": [{"question": query}, {"topic": topic}]}))
            
    def add_wiki_documents(self, title: str, wiki_chunks: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        result = self.get_wiki_documents(title)
        if len(result) == 0:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(wiki_chunks))))
            metas=[{"title": title, "source": "add_wiki_documents"}]*len(wiki_chunks)
            tqdm(self.db.add(ids=ids, documents=wiki_chunks, metadatas=metas), desc="Generate wiki embeddings")

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_with_wiki_passages(self, query: str, title: str, passages: list[str]):
        return self.generate_answer(query, where={"title": title}, passages=passages)
    
    def get_wiki_documents(self, title: Optional[str] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        if title is None:
            return self.stored_result(self.db.get(where={"source": "add_wiki_document"}))
        else:
            return self.stored_result(self.db.get(where={"title": title}))

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_documents_list(self, query: str, max_sources: int = 5000, where: Optional[dict] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.stored_result(
            self.db.query(query_texts=[query], 
                          n_results=max_sources, 
                          where=where), 
            is_query = True)

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_exchanges_csv(self, query: str):
        return self.generate_answer(query, max_sources=100, where={"source": "exchanges.csv"})

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_answer(self, query: str, max_sources: int = 10, 
                        where: Optional[dict] = None, passages: Optional[list[str]] = None):
        stored = self.get_documents_list(query, max_sources, where)
        query_oneline = query.replace("\n", " ")
        prompt = f"""You're an expert writer. You understand how to interpret html and markdown. You will accept the
        question below and answer based only on the passages. Never mention the passages in your answers. Be sure to 
        respond in concise sentences. Include all relevant background information when possible. If a passage is not 
        relevant to the answer you must ignore it. If no passage answers the question respond with: I don't know.

        QUESTION: {query_oneline}
        
        """
        # Add the retrieved documents to the prompt.
        stored_docs = [passage.docs for passage in stored]
        for passage in stored_docs if passages is None else stored_docs + passages:
            passage_oneline = passage.replace("\n", " ")
            prompt += f"PASSAGE: {passage_oneline}\n"
        # Generate the response.
        response = api.retriable(
            self.client.models.generate_content,
            model=api(Gemini.Model.GEN),
            config=self.config_temp,
            contents=prompt)
        # Check for generated code and store in memory.
        content = response.candidates[0].content
        if len(content.parts) > 1 and content.parts[0].executable_code:
            memory.append_code(prompt, content.parts)
        return response

    def stored_result(self, result, is_query: bool = False) -> list[VectorStoreResult]:
        try:
            results = []
            if len(result["documents"]) == 0:
                return results
            if isinstance(result["documents"][0], list):
                for i in range(len(result["documents"][0])):
                    obj = VectorStoreResult(docs=result["documents"][0][i],
                                            dist=result["distances"][0][i] if is_query else None,
                                            meta=result["metadatas"][0][i],
                                            store_id=result["ids"][0][i])
                    results.append(obj)
            else:
                results.append(
                    VectorStoreResult(docs=result["documents"][0],
                                      dist=result["distances"][0] if is_query else None,
                                      meta=result["metadatas"][0],
                                      store_id=result["ids"][0]))
            return results
        except Exception as e:
            raise e

## Wikipedia Search Tool

In [16]:
# An implementation of Wiki-Grounding Generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by similarity to topic
# - retrieve existing groundings by similarity to topic
class WikiGroundingGenerator:   
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl
        with warnings.catch_warnings():
            warnings.simplefilter("ignore") # suppress beta-warning
            self.splitter = HTMLSemanticPreservingSplitter(
                headers_to_split_on=[("h2", "Main Topic"), ("h3", "Sub Topic")],
                separators=["\n\n", "\n", ". ", "! ", "? "],
                max_chunk_size=Gemini.Const.ChunkMax(),
                chunk_overlap=50,
                preserve_links=True,
                preserve_images=True,
                preserve_videos=True,
                preserve_audio=True,
                elements_to_preserve=["table", "ul", "ol", "code"],
                denylist_tags=["script", "style", "head"],
                custom_handlers={"code": self.code_handler},
            )

    def generate_answer(self, query: str, topic: str):
        stored = self.rag.get_wiki_documents(topic)
        if len(stored) > 0:
            return self.rag.generate_with_wiki_passages(query, topic, [chunk.docs for chunk in stored]).text
        else:
            pages = wikipedia.search(topic + " company")
            if len(pages) > 0:
                p_topic_match = 0.80
                for i in range(len(pages)):
                    if tqdm(api.similarity([topic + " company", pages[i]]) > p_topic_match, 
                            desc= "Score wiki search by similarity to topic"):
                        headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:142.0) Gecko/20100101 Firefox/142.0'}
                        request = requests.get(f"https://en.wikipedia.org/wiki/{pages[i]}", headers=headers)
                        if request.status_code != 200:
                            print(f"wiki get() returned status code {request.status_code}")
                        chunks = [chunk.page_content for chunk in self.splitter.split_text(request.text)]
                        self.rag.add_wiki_documents(topic, chunks)
                        return self.rag.generate_with_wiki_passages(query, topic, chunks).text
            return StopGeneration().result

    def code_handler(self, element: Tag) -> str:
        data_lang = element.get("data-lang")
        code_format = f"<code:{data_lang}>{element.get_text()}</code>"
        return code_format

## Google Search Tool

In [17]:
# An implementation of Search-Grounding Generation.
# - using gemini-2.0-flash with GoogleSearch tool for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by exact match to topic
# - retrieve existing groundings by similarity to topic
class SearchGroundingGenerator:
    config_ground = types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        temperature=0.0
    )
    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl

    def generate_answer(self, query: str, topic: str):
        stored = self.rag.get_grounding_documents(query, topic)
        if len(stored) > 0:
            for i in range(len(stored)):
                meta_q = stored[i].meta["question"]
                p_ground_match = 0.95 # This can be really high ~ 95-97%
                if tqdm(api.similarity([query, meta_q]) > p_ground_match,
                        desc="Score similarity to stored grounding"):
                    return ast.literal_eval(stored[i].docs)
        return self.get_grounding(query, topic)

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_grounding(self, query: str, topic: str):
        contents = [types.Content(role="user", parts=[types.Part(text=query)])]
        contents += f"""
        You're a search assistant that provides grounded answers to questions about {topic}. You will provide only 
        results that discuss {topic}. Be brief and specific in answering and omit extra details.
        If an answer is not possible respond with: I don't know."""
        response = api.retriable(self.client.models.generate_content, 
                                 model=api(Gemini.Model.GEN), 
                                 config=self.config_ground, 
                                 contents=contents)
        if response.candidates[0].grounding_metadata.grounding_supports is not None:
            if self.is_consistent(query, topic, response.text):
                self.rag.add_grounded_document(query, topic, response)
                return response.text 
        return StopGeneration().result # Empty grounding supports or not consistent in response

    def is_consistent(self, query: str, topic: str, model_response: str) -> bool:
        topic = topic.replace("'", "")
        id_strs = topic.split()
        if len(id_strs) == 1:
            matches = re.findall(rf"{id_strs[0]}[\s,.]+\S+", query)
            if len(matches) > 0:
                topic = matches
        compound_match = re.findall(rf"{id_strs[0]}[\s,.]+\S+", model_response)
        model_response = model_response.replace("'", "")
        if len(compound_match) == 0 and topic in model_response:
            return True # not a compound topic id and exact topic match
        for match in compound_match:
            if topic not in match:
                return False
        return True # all prefix matches contained topic

## Rest API Tool and Helpers

In [18]:
# Rest api-helpers to manage request-per-minute limits.
# - define an entry for each endpoint limit
# - init rest tool with limits to create blocking queues
# - apply a limit to requests with rest_tool.try_url
class ApiLimit(Enum):
    FINN = "finnhub.io",60
    POLY = "polygon.io",5 # (id_url,rpm)

class BlockingUrlQueue:
    on_cooldown = False
    cooldown = None
    cooldown_start = None
    
    def __init__(self, rest_fn: Callable, per_minute: int):
        self.per_minute_max = per_minute
        self.quota = per_minute
        self.rest_fn = rest_fn

    def push(self, rest_url: str):
        if not self.on_cooldown:
            self.cooldown = Timer(60, self.reset_quota)
            self.cooldown.start()
            self.cooldown_start = time.time()
            self.on_cooldown = True
        if self.quota > 0:
            self.quota -= 1
            time.sleep(0.034) # ~30 requests per second
            return self.rest_fn(rest_url)
        else:
            print(f"limited {self.per_minute_max}/min, waiting {self.limit_expiry()}s")
            time.sleep(max(self.limit_expiry(),0.5))
            return self.push(rest_url)

    def reset_quota(self):
        self.quota = self.per_minute_max
        self.on_cooldown = False
        self.cooldown_start = None

    def limit_expiry(self):
        if self.cooldown_start:
            return max(60-(time.time()-self.cooldown_start),0)
        return 0

In [19]:
# An implementation of Rest-Grounding Generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - reduce long-context by chunked pre-processing
class RestGroundingGenerator:    
    limits = None

    def __init__(self, rag_impl, with_limits: bool):
        self.rag = rag_impl
        if with_limits:
            self.limits = {}
            for rest_api in ApiLimit:
                self.limits[rest_api.value[0]] = BlockingUrlQueue(self.get, rest_api.value[1])

    def get_limit(self, rest_api: ApiLimit) -> Optional[BlockingUrlQueue]:
        return self.limits[rest_api.value[0]] if self.limits else None

    def get(self, url: str) -> Optional[str]:
        try:
            headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:142.0) Gecko/20100101 Firefox/142.0'}
            request = requests.get(url, headers=headers)
            if request.status_code != requests.codes.ok:
                print(f"the endpoint returned status {request.status_code}")
            return request.text
        except Exception as e:
            raise e

    def basemodel(self, data: str, schema: BaseModel, from_lambda: bool = False) -> Optional[BaseModel]:
        try:
            if from_lambda:
                return schema(results=json.loads(data))
            return schema.model_validate_json(data)
        except Exception as e:
            raise e

    def dailycandle(self, data: str) -> Optional[DailyCandle]:
        try:
            candle = json.loads(data)
            if "from" not in candle:
                raise ValueError("not a dailycandle / missing value for date")
            agg = self.basemodel(data, Aggregate)
            return DailyCandle(from_date=candle["from"], 
                               status=agg.status.value, 
                               symbol=agg.symbol, 
                               open=agg.open, 
                               high=agg.high, 
                               low=agg.low, 
                               close=agg.close, 
                               volume=agg.volume, 
                               otc=agg.otc, 
                               preMarket=agg.preMarket, 
                               afterHours=agg.afterHours)
        except Exception as e:
            raise e

    @retry.Retry(timeout=600)
    def try_url(self, url: str, schema: BaseModel, as_lambda: bool, with_limit: Optional[BlockingUrlQueue],
                success_fn: Callable, *args, **kwargs):
        try:
            if self.limits is None:
                data = self.get(url)
            elif with_limit:
                data = with_limit.push(url)
            if schema is DailyCandle:
                model = self.dailycandle(data)
            else:
                model = self.basemodel(data, schema, as_lambda)
        except Exception as e:
            try:
                print(f"try_url exception: {e}")
                if issubclass(schema, RestResultPoly):
                    return success_fn(*args, **kwargs, result=self.basemodel(data, RestResultPoly))
            except Exception as not_a_result:
                print(not_a_result)
            return StopGeneration()
        else:
            return success_fn(*args, **kwargs, model=model)

    def get_symbol_matches(self, with_content, by_name: bool, model: SymbolResult):
        matches = []
        max_failed_match = model.count if not by_name else 3
        p_desc_match = 0.92
        p_symb_match = 0.95
        if model.count > 0:
            for obj in tqdm(model.get(), desc="Score similarity to query"):
                if max_failed_match > 0:
                    desc = [with_content["q"].upper(), obj.description.split("-", -1)[0]]
                    symb = [with_content["q"].upper(), obj.symbol]
                    if by_name and api.similarity(desc) > p_desc_match: 
                        matches.append(obj.symbol)
                    elif not by_name and api.similarity(symb) > p_symb_match:
                        matches.append(obj.description)
                        max_failed_match = 0
                    else:
                        max_failed_match -= 1
        if len(matches) > 0:
            self.rag.add_api_document(with_content["query"], matches, with_content["q"], "get_symbol_1")
            return matches
        return StopGeneration().result

    def get_quote(self, with_content, model: Quote):
        quote = model.model_dump_json()
        self.rag.add_quote_document(with_content["query"], quote, with_content["symbol"], model.t, "get_quote_1")
        return quote

    def parse_financials(self, with_content, model: BasicFinancials):
        metric = list(model.metric.items())
        chunks = []
        # Chunk the metric data.
        for i in range(0, len(metric), Gemini.Const.MetricBatch()):
            batch = metric[i:i + Gemini.Const.MetricBatch()]
            chunks.append({"question": with_content["query"], "answer": batch})
        # Chunk the series data.
        for key in model.series.keys():
            series = list(model.series[key].items())
            for s in series:
                if api.token_count(s) <= Gemini.Const.ChunkMax():
                    chunks.append({"question": with_content["query"], "answer": s})
                else:
                    k = s[0]
                    v = s[1]
                    for i in range(0, len(v), Gemini.Const.SeriesBatch()):
                        batch = v[i:i + Gemini.Const.SeriesBatch()]
                        chunks.append({"question": with_content["query"], "answer": {k: batch}})
        self.rag.add_rest_chunks(chunks, topic=with_content["symbol"], source="get_financials_1")
        return chunks

    def parse_news(self, with_content, model: NewsResultFinn):
        if model.count > 0:
            metas = []
            for digest in model.get():
                pub_date = datetime.fromtimestamp(digest.datetime, tz=GeneratedEvent.tz()).strftime("%Y-%m-%d")
                metas.append({"publisher": digest.source,
                              "published_est": parse(pub_date).timestamp(),
                              "news_id": digest.id,
                              "related": digest.related})
            self.rag.add_rest_chunks(model.get(), topic=with_content["symbol"], source="get_news_1",
                                     ids=[f"{digest.id}+news" for digest in model.get()],
                                     meta_opt=metas, is_update=False)
            return [digest.summary().model_dump_json() for digest in model.get()]
        return StopGeneration().result

    def parse_news(self, with_content, model: Optional[NewsResultPoly] = None,
                   result: Optional[RestResultPoly] = None) -> tuple[list, str]: # list of summary, next list url
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            metas = []
            for news in model.get():
                pub_date = parse(news.published_utc).strftime("%Y-%m-%d")
                metas.append({"publisher": news.publisher.name,
                              "published_utc": parse(pub_date).timestamp(),
                              "news_id": news.id,
                              "related": json.dumps(news.tickers),
                              "keywords": json.dumps(news.keywords)})
            self.rag.add_rest_chunks(model.get(), topic=with_content["ticker"], source="get_news_2",
                                     ids=[news.id for news in model.get()],
                                     meta_opt=metas, is_update=False)
            return [news.summary().model_dump_json() for news in model.get()], model.next_url
        elif result:
            return result.model_dump_json()

    def parse_daily_candle(self, with_content, model: Optional[DailyCandle] = None,
                           result: Optional[RestResultPoly] = None):
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            self.rag.add_rest_chunks(
                chunks=[model],
                topic=with_content["stocksTicker"],
                source="daily_candle_2",
                meta_opt=[{"from_date": model.from_date, "adjusted": with_content["adjusted"]}])
            return model
        elif result:
            return result

    def parse_custom_candle(self, with_content, model: Optional[CustomCandle] = None,
                            result: Optional[RestResultPoly] = None):
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            metas = [{
                "timespan": with_content["timespan"],
                "adjusted": with_content["adjusted"],
                "from": with_content["from"],
                "to": with_content["to"]}]*model.count
            candles = [candle.model_dump_json() for candle in model.get()]
            self.rag.add_rest_chunks(
                chunks=candles,
                topic=with_content["stocksTicker"],
                source="custom_candle_2",
                meta_opt=metas)
            return candles
        elif result:
            return result.model_dump_json()

    def parse_overview(self, with_content, model: OverviewResult):
        overview = [model.get().model_dump_json()]
        self.rag.add_rest_chunks(chunks=overview, topic=with_content["ticker"], source="ticker_overview_2")
        return overview

    def parse_trends(self, with_content, model: TrendsResult):
        if model.count > 0:
            metas = [{"period": trend.period} for trend in model.get()]
            trends = [trend.model_dump_json() for trend in model.get()]
            self.rag.add_rest_chunks(trends, topic=with_content["symbol"], source="trends_1", meta_opt=metas)
            return trends
        return StopGeneration().result

    def augment_market_status(self, with_id: Optional[str], model: MarketStatusResult):
        if model.get().holiday != MarketSession.NA.value:
            self.rag.set_holiday_event(model.get().exchange)
        events = self.rag.generated_events(model.get().exchange)
        model.get().session = events.session()
        model.get().isOpen = events.is_open()
        meta = {"exchange": model.get().exchange,
                "last_close": events.last_close,
                "pre_open": events.pre_open,
                "reg_open": events.reg_open,
                "reg_close": events.reg_close,
                "post_close": events.post_close,
                "timestamp": events.timestamp }
        self.rag.add_rest_chunks([model.get()],
                                 topic="market_status",
                                 source="get_market_status_1",
                                 ids=[with_id] if with_id else None,
                                 meta_opt=[meta])
        return model.get().model_dump_json()

    def get_symbol(self, content, by_name: bool = True):
        return self.try_url(
            f"https://finnhub.io/api/v1/search?q={content['q']}&exchange={content['exchange']}&token={FINNHUB_API_KEY}",
            schema=SymbolResult,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.get_symbol_matches,
            with_content=content,
            by_name=by_name)

    def get_current_price(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/quote?symbol={content['symbol']}&token={FINNHUB_API_KEY}",
            schema=Quote,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.get_quote,
            with_content=content)

    def get_market_status(self, content, store_id: Optional[str] = None):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/market-status?exchange={content['exchange']}&token={FINNHUB_API_KEY}",
            schema=MarketStatusResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.augment_market_status,
            with_id=store_id)

    def get_peers(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/peers?symbol={content['symbol']}&grouping={content['grouping']}&token={FINNHUB_API_KEY}",
            schema=PeersResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=lambda model: model)

    def get_basic_financials(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/metric?symbol={content['symbol']}&metric={content['metric']}&token={FINNHUB_API_KEY}",
            schema=BasicFinancials,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_financials,
            with_content=content)

    def get_news_simple(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/company-news?symbol={content['symbol']}&from={content['from']}&to={content['to']}&token={FINNHUB_API_KEY}",
            schema=NewsResultFinn,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_news,
            with_content=content)

    def get_news_tagged(self, content):
        next_url = f"https://api.polygon.io/v2/reference/news?ticker={content['ticker']}&published_utc.gte={content['published_utc.gte']}&published_utc.lte={content['published_utc.lte']}&order={content['order']}&limit={content['limit']}&sort={content['sort']}&apiKey={POLYGON_API_KEY}"
        news = []
        while True:
            news_list, next_url = self.try_url(
                next_url,
                schema=NewsResultPoly,
                as_lambda=False,
                with_limit=self.get_limit(ApiLimit.POLY),
                success_fn=self.parse_news,
                with_content=content)
            news += news_list
            if next_url is None:
                break
            next_url += f"&apiKey={POLYGON_API_KEY}"
        return news

    def get_daily_candle(self, content):
        return self.try_url(
            f"https://api.polygon.io/v1/open-close/{content['stocksTicker']}/{content['date']}?adjusted={content['adjusted']}&apiKey={POLYGON_API_KEY}",
            schema=DailyCandle,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_daily_candle,
            with_content=content)

    def get_custom_candle(self, content):
        return self.try_url(
            f"https://api.polygon.io/v2/aggs/ticker/{content['stocksTicker']}/range/{content['multiplier']}/{content['timespan']}/{content['from']}/{content['to']}?adjusted={content['adjusted']}&sort={content['sort']}&limit={content['limit']}&apiKey={POLYGON_API_KEY}",
            schema=CustomCandle,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_custom_candle,
            with_content=content)

    def get_overview(self, content):
        return self.try_url(
            f"https://api.polygon.io/v3/reference/tickers/{content['ticker']}?apiKey={POLYGON_API_KEY}",
            schema=OverviewResult,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_overview,
            with_content=content)

    def get_trends_simple(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/recommendation?symbol={content['symbol']}&token={FINNHUB_API_KEY}",
            schema=TrendsResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_trends,
            with_content=content)

# Instantiate the Tools

<span style="font-size:18px;">
Let's load some test data and see what the RAG can do. The test data is a CSV file containing stock market exchange data. It includes the market id code, name, locale, and operating hours. The import will use CSVLoader from <code>langchain-community</code> to parse the exchange data into Documents that our RAG can ingest.
</span>

In [20]:
# Instantiate tools and load the exchange data from source csv.
# - Identifies exchanges by a 1-2 letter code which can be used to filter response data.
# - Also maps the exchange code to exchange details.
try:
    df = pandas.read_csv("/kaggle/input/exchanges/exchanges_src.csv")
except FileNotFoundError as e:
    df = pandas.read_csv("exchanges_src.csv") # local run
df = df.drop(["close_date"], axis=1).fillna("")
df.to_csv("exchanges.csv", index=False)
exchanges = CSVLoader(file_path="exchanges.csv", encoding="utf-8", csv_args={"delimiter": ","}).load()

# Prepare a RAG tool for use and add the exchange data.
tool_rag = RetrievalAugmentedGenerator(api.client, "finance")
tool_rag.add_documents_list(exchanges)
tool_rag.generated_events("US")

# Prepare a the grounding tools for use.
tool_wiki = WikiGroundingGenerator(api.client, tool_rag)
tool_ground = SearchGroundingGenerator(api.client, tool_rag)
tool_rest = RestGroundingGenerator(tool_rag, with_limits=True)

Generate document embedding: 0it [00:00, ?it/s]
Generate US->MarketEvent.POST_CLOSE: 100%|██████████| 1/1 [00:01<00:00,  1.96s/it]


<span style="font-size:18px;">
Now that the data is loaded lets ask our RAG to perform some augmenting. We can ask it to perform all sorts of useful tasks. We'll generate some useful reusable data structures and check to make sure it can answer important questions. The exchanges all have id's which are used to filter the realtime data. So we'll make sure the RAG know how to create this mapping. We'll also check it's awareness of operating hours. After all, Essy, doesn't mindlessly hammer away at api's when no new data is available.
</span>

In [62]:
# The RAG tool is a helpful expert.

response = tool_rag.get_exchanges_csv(
    """Give me a dictionary in string form. It must contain key:value pairs mapping 
    exchange code to name. Just the dictionary string in pretty form.""")
print(response.text)

response = tool_rag.get_exchanges_csv(
    """What is the Germany exchange code? Return only the exchange codes as a simple 
    comma separated value that I can copy.""")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv("What are the Germany exchanges and thier corresponding exchange codes?")
print(response.text, "\n")

response = tool_rag.generate_answer("What are Google's stock ticker symbols?")
print(response.text, "\n")

response = tool_rag.generate_answer("What is Facebook's stock ticker symbol?")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv("What are the US exchange operating hours?")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv(
    f"""Answer based on your knowledge of exchange operating hours.
    Do not answer in full sentences. Omit all chat and provide the answer only.
    The fields pre_market and post_market both represent extended operating hours.

    The current date and time: {datetime.now(GeneratedEvent.tz()).strftime('%c')}

    Weekdays are: Mon, Tue, Wed, Thu, Fri.
    On weekdays all exchanges open after pre-market and regular hours.
    On weekdays all exchanges close after regular and post-market hours.
    
    Weekends are: Sat, Sun.
    Always exclude weekends from exchange operating hours.
    A list of holidays in date format mm-dd-yyyy: {tool_rag.holidays["US"]}
    Always exclude holidays from exchange operating hours.
    When the answer is a holiday use the prior weekday for close.
    When the answer is a holiday use the next weekday for open.
    
    Consider the US exchange's operating hours.
    Provide the most recent weekday's close including post_market hours.
    
    Answer with a date that uses this format: '%a %b %d %X %Y'.""")
print(response.candidates[0].content.parts[-1].text)

```
{
    "VN": "Vietnam exchanges including HOSE, HNX and UPCOM",
    "AD": "ABU DHABI SECURITIES EXCHANGE",
    "US": "US exchanges (NYSE, Nasdaq)",
    "CO": "OMX NORDIC EXCHANGE COPENHAGEN A/S",
    "QA": "QATAR EXCHANGE",
    "BA": "BOLSA DE COMERCIO DE BUENOS AIRES",
    "MX": "BOLSA MEXICANA DE VALORES (MEXICAN STOCK EXCHANGE)",
    "PR": "PRAGUE STOCK EXCHANGE",
    "HK": "HONG KONG EXCHANGES AND CLEARING LTD",
    "CA": "Egyptian Stock Exchange",
    "AX": "ASX - ALL MARKETS",
    "SX": "DEUTSCHE BOERSE Stoxx",
    "KQ": "KOREA EXCHANGE (KOSDAQ)",
    "DB": "DUBAI FINANCIAL MARKET",
    "PM": "Philippine Stock Exchange",
    "KS": "KOREA EXCHANGE (STOCK MARKET)",
    "ST": "NASDAQ OMX NORDIC STOCKHOLM",
    "DU": "BOERSE DUESSELDORF",
    "TL": "NASDAQ OMX TALLINN",
    "AT": "ATHENS EXCHANGE S.A. CASH MARKET",
    "SW": "SWISS EXCHANGE",
    "LS": "NYSE EURONEXT - EURONEXT LISBON",
    "SI": "SINGAPORE EXCHANGE",
    "RG": "NASDAQ OMX RIGA",
    "CR": "CARACAS STOCK EXCHANGE"

<span style="font-size:18px;">
Excellent! Though, despite my best effort I could not convince Gemini to apply date correction (during chaining) based on holiday. It simply wasn't stable enough to be useful. I would either have to add a holiday data set, or (what I chose) apply a quick temporary fix. A real-time API endpoint may fail due to a holiday being selected as the date. If that happens I'll just retry Thursday if the failure happened on Friday, likewise choosing Friday if the failure happened on Monday. Crude but simple for foundational purposes.
</span>

# Declaring the Function Calling Metadata

<span style="font-size:18px;">
Our Function Calling expert will chain together the other experts we've implemented thus far. It also provides the final response through augmentation. This time using the tools as a source of grounding truth. It'd like to say it's all truth organised by topic and other metadata. It's still a precarious situation if Essy incidently chains into mining data on another topic. We want Amazon to be the owner of MGM Studio's not MGM Resorts International. We also don't want a summary to include another company unless that company is a peer.
</span>

<span style="font-size:18px;">
The function calling metadata is thus extremely important. It needs to combine our other experts with the real-time api's data. Essy will use two API providers as sources of finance data. The primary motivation being that each provider has limits in their own way, yet both are useful in their own own way. This is useful anywhere you need a broad spectrum of sources of truth. At metadata creation I'll adopt the naming convention of appending the provider (if any) id. This helps keep functions more understandable when you know which provider you're dealing with.
</span>

In [22]:
# Declare callable functions using OpenAPI schema.
decl_get_symbol_1 = types.FunctionDeclaration(
    name="get_symbol_1",
    description="""Search for the stock ticker symbol of a given company, security, isin or cusip. Each ticker
                   entry provides a description, symbol, and asset type. If this doesn't help you should try 
                   calling get_wiki_tool_response next.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The company, security, isin or cusip to search for a symbol."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["q", "exchange", "query"]
    }
)

decl_get_symbols_1 = types.FunctionDeclaration(
    name="get_symbols_1",
    description="""List all supported symbols and tickers. The results are filtered by exchange code.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter the results."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["exchange", "query"]
    }
)

decl_get_name_1 = types.FunctionDeclaration(
    name="get_name_1",
    description="""Search for the name associated with a stock ticker or symbol's company, security, isin or cusip. 
    Each ticker entry provides a description, matching symbol, and asset type.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The symbol or ticker to search for."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "company": {
                "type": "string",
                "description": "The company you're searching for."
            }
        },
        "required": ["q", "exchange", "query", "company"]
    }
)

decl_get_symbol_quote_1 = types.FunctionDeclaration(
    name="get_symbol_quote_1",
    description="""Search for the current price or quote of a stock ticker or symbol. The response is
                   provided in json format. Each response contains the following key-value pairs:
                   
                   c: Current price,
                   d: Change,
                  dp: Percent change,
                   h: High price of the day,
                   l: Low price of the day,
                   o: Open price of the day,
                  pc: Previous close price,
                   t: Epoch timestamp of price in seconds.

                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol for a company, security, isin, or cusip." 
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "exchange": {
                "type": "string",
                "description": "The exchange code used to filter quotes. This must always be 'US'."
            }
        },
        "required": ["symbol", "query", "exchange"]
    }
)

decl_get_local_datetime = types.FunctionDeclaration(
    name="get_local_datetime",
    description="""Converts an array of timestamps from epoch time to the local timezone format. The result is an array
                   of date and time in locale appropriate format. Suitable for use in a locale appropriate response.
                   Treat this function as a vector function. Always prefer to batch timestamps for conversion. Use this
                   function to format date and time in your responses.""",
    parameters={
        "type": "object",
        "properties": {
            "t": {
                "type": "array",
                "description": """An array of timestamps in seconds since epoch to be converted. The order of
                                  timestamps matches the order of conversion.""",
                "items": {
                    "type": "integer"
                }
            }
        },
        "required": ["t"]
    }
)

decl_get_market_status_1 = types.FunctionDeclaration(
    name="get_market_status_1",
    description="""Get the current market status of global exchanges. Includes whether exchanges are open or closed.  
                   Also includes holiday details if applicable. The response is provided in json format. Each response 
                   contains the following key-value pairs:

                   exchange: Exchange code,
                   timezone: Timezone of the exchange,
                    holiday: Holiday event name, or null if it's not a holiday,
                     isOpen: Whether the market is open at the moment,
                          t: Epoch timestamp of status in seconds (Eastern Time),
                    session: The market session can be 1 of the following values: 
                    
                    pre-market,regular,post-market when open, or null if closed.
                    
                    Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_market_session_1 = types.FunctionDeclaration(
    name="get_market_session_1",
    description="Get the current market session of global exchanges.",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_company_peers_1 = types.FunctionDeclaration(
    name="get_company_peers_1",
    description="""Search for a company's peers. Returns a list of peers operating in the same country and in the same
                   sector, industry, or subIndustry. Each response contains the following key-value pairs: 
                   
                   symbol: The company's stock ticker symbol, 
                   peers: A list containing the peers.
                   
                   Each peers entry contains the following key-value pairs:
                   
                   symbol: The peer company's stock ticker symbol, 
                   name: The peer company's name.
                   
                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol of a company to obtain peers."
            },
            "grouping": {
                "type": "string",
                "description": """This parameter may be one of the following values: sector, industry, subIndustry.
                                  Always use subIndustry unless told otherwise."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "grouping", "exchange", "query"]
    }
)

decl_get_exchange_codes_1 = types.FunctionDeclaration(
    name="get_exchange_codes_1",
    description="""Get a dictionary mapping all supported exchange codes to their names."""
)

decl_get_exchange_code_1 = types.FunctionDeclaration(
    name="get_exchange_code_1",
    description="""Search for the exchange code to use when filtering by exchange. The result will be one or
                   more exchange codes provided as a comma-separated string value.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "Specifies which exchange code to search for."
            }
        },
        "required": ["q"]
    }
)

decl_get_financials_1 = types.FunctionDeclaration(
    name="get_financials_1",
    description="""Get company basic financials such as margin, P/E ratio, 52-week high/low, etc. Parse the response for 
                   key-value pairs in json format and interpret their meaning as stock market financial indicators.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "metric": {
                "type": "string",
                "description": "It must always be declared as the value 'all'"
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "metric", "query"]
    }
)

decl_get_daily_candlestick_2 = types.FunctionDeclaration(
    name="get_daily_candlestick_2",
    description="""Get a historical daily stock ticker candlestick / aggregate bar (OHLC). 
                   Includes historical daily open, high, low, and close prices. Also includes historical daily trade
                   volume and pre-market/after-hours trade prices. It does not provide today's data until after 
                   11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "date": {
                "type": "string",
                "format": "date-time",
                "description": """The date of the requested candlestick in format YYYY-MM-DD."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicates if the results should be adjusted for splits.
                                  Use true unless told otherwise."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "date", "adjusted", "exchange", "query"]
    },
)

decl_get_company_news_1 = types.FunctionDeclaration(
    name="get_company_news_1",
    description="Retrieve the most recent news articles related to a specified ticker.",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company.",
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be older than the parameter 'to'."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be more recent than the parameter 'from'. The
                                  default value is today's date."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "from", "to", "query"]
    },
)

decl_get_custom_candlestick_2 = types.FunctionDeclaration(
    name="get_custom_candlestick_2",
    description="""Get a historical stock ticker candlestick / aggregate bar (OHLC) over a custom date range and 
                   time interval in Eastern Time. Includes historical open, high, low, and close prices. Also 
                   includes historical daily trade volume and pre-market/after-hours trade prices. It does not
                   include today's open, high, low, or close until after 11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "multiplier": {
                "type": "integer",
                "description": "This must be included and equal to 1 unless told otherwise."
            },
            "timespan": {
                "type": "string",
                "description": """The size of the candlestick's time window. This is allowed to be one of the following:
                                  second, minute, hour, day, week, month, quarter, or year. The default value is day."""
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'to'."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'from'. The 
                                  default is one weekday before get_last_market_close.
                                  Replace more recent dates with the default."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicates if the results should be adjusted for splits.
                                  Use true unless told otherwise."""
            },
            "sort": {
                "type": "string",
                "description": """This must be included. May be one of asc or desc. asc will sort by timestmap in 
                                  ascending order. desc will sort by timestamp in descending order."""
            },
            "limit": {
                "type": "integer",
                "description": """Set the number of base aggregates used to create this candlestick. This must be 5000 
                                  unless told to limit base aggregates to something else."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "multiplier", "timespan", "from", "to", "adjusted", "sort", "limit", "query"]
    },
)

decl_get_last_market_close = types.FunctionDeclaration(
    name="get_last_market_close",
    description="""Get the last market close of the specified exchange in Eastern Time. The response has already
                   been converted by get_local_datetime so this step should be skipped.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_ticker_overview_2 = types.FunctionDeclaration(
    name="get_ticker_overview_2",
    description="""Retrieve comprehensive details for a single ticker symbol. It's a deep look into a company’s 
    fundamental attributes, including its primary exchange, standardized identifiers (CIK, composite FIGI, 
    share class FIGI), market capitalization, industry classification, and key dates. Also includes branding assets in
    the form of icons and logos.
    """,
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol of a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "query"]
    }
)

decl_get_recommendation_trends_1 = types.FunctionDeclaration(
    name="get_recommendation_trends_1",
    description="""Get the latest analyst recommendation trends for a company.
                The data includes the latest recommendations as well as historical
                recommendation data for each month. The data is classified according
                to these categories: strongBuy, buy, hold, sell, and strongSell.
                The date of a recommendation indicated by the value of 'period'.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "query"]
    }
)

decl_get_news_with_sentiment_2 = types.FunctionDeclaration(
    name="get_news_with_sentiment_2",
    description="""Retrieve the most recent news articles related to a specified ticker. Each article includes 
                   comprehensive coverage. Including a summary, publisher information, article metadata, 
                   and sentiment analysis.""",
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "published_utc.gte": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'published_utc.lte'. 
                                  The default value is one-month ago from today's date."""
            },
            "published_utc.lte": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'published_utc.gte'.
                                  The default is one weekday prior to get_last_market_close (excluding weekends).
                                  Replace more recent dates with the default."""
            },
            "order": {
                "type": "string",
                "description": """Must be desc for descending order, or asc for ascending order.
                                  When order is not specified the default is descending order.
                                  Ordering will be based on the parameter 'sort'."""
            },
            "limit": {
                "type": "integer",
                "description": """This must be included and equal to 1000 unless told otherwise."""
            },
            "sort": {
                "type": "string",
                "description": """The sort field used for ordering. This value must
                                  always be published_utc."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["limit", "ticker", "published_utc.gte", "published_utc.lte", "order", "sort", "query"]
    }
)

decl_get_rag_tool_response = types.FunctionDeclaration(
    name="get_rag_tool_response",
    description="""A database containing useful financial information. Always check here for answers first.""",
    parameters={
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "A question needing an answer. Asked as a simple string."
            }
        }
    }
)

decl_get_wiki_tool_response = types.FunctionDeclaration(
    name="get_wiki_tool_response",
    description="""Answers questions that still have unknown answers. Retrieve a wiki page related to a company, 
                   product, or service. Each web page includes detailed company information, financial indicators, 
                   tickers, symbols, history, and products and services.""",
    parameters={
        "type": "object",
        "properties": {
            "id": {
                "type": "string",
                "description": "The question's company or product. Just the name and no other details."
            },
            "q": {
                "type": "string",
                "description": "The complete, unaltered, query string."
            }
        },
        "required": ["id", "q"]
    }
)

decl_get_search_tool_response = types.FunctionDeclaration(
    name="get_search_tool_response",
    description="Answers questions that still have unknown answers. Use it after checking all your other tools.",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "The question needing an answer. Asked as a simple string."
            },
            "id": {
                "type": "string",
                "description": "The question's company or product. In one word. Just the name and no other details."
            }
        },
        "required": ["q", "id"]
    }
)

# Implementing the Function Calling Expert

<span style="font-size:18px;">
One downside of this part being the main part was the lack of time to refactor this part more. Our formative Essy implements as much useful data from two finacial APIs. In order to use it you will need to declare secrets for <a class="anchor-link" href="https://finnhub.io/dashboard">Finnhub</a> and <a class="anchor-link" href="https://polygon.io/dashboard">Polygon</a> finance APIs. Register at their respective sites for your free API key. Then import the secret using the same method as how you setup Google's API key.
</span>

## Callable Functions and Handler

In [23]:
# Implement the callable functions and the function handler.

def ask_rag_tool(content):
    return tool_rag.generate_answer(content["question"]).text

def ask_wiki_tool(content):
    return tool_wiki.generate_answer(content["q"], content["id"])

def ask_search_tool(content):
    return tool_ground.generate_answer(content["q"], content["id"])

def get_exchange_codes_1(content):
    return tool_rag.get_exchange_codes()

def get_exchange_code_1(content):
    return tool_rag.get_exchange_codes(with_query=content)
    
def last_market_close(content):
    return tool_rag.last_market_close(content["exchange"])
    
def get_symbol_1(content, by_name: bool = True):
    stored = tool_rag.get_api_documents(content["query"], content["q"], "get_symbol_1")
    if len(stored) == 0:
        return tool_rest.get_symbol(content, by_name)
    return json.loads(stored[0].docs)

def get_symbols_1(content):
    return None # todo

def get_name_1(content):
    return get_symbol_1(content, by_name = False)

def get_quote_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "get_quote_1")
    if tool_rag.generated_events(content["exchange"]).is_open():
        return get_current_price_1(content)
    elif len(stored) > 0:
        last_close = parse(tool_rag.last_market_close(content["exchange"])).timestamp()
        for quote in stored:
            if quote.meta["timestamp"] >= last_close:
                return [quote.docs for quote in stored]
    return get_current_price_1(content)

def get_current_price_1(content):
    return tool_rest.get_current_price(content)

def get_market_status_1(content):
    stored, has_update = tool_rag.get_market_status(content['exchange'])
    if has_update:
        with_id = stored[0].store_id if len(stored) > 0 else None
        return tool_rest.get_market_status(content, with_id)
    return stored[0].docs

def get_session_1(content):
    return json.loads(get_market_status_1(content))["session"]

def get_peers_1(content):
    stored = tool_rag.get_peers_document(content["query"], content["symbol"], content['grouping'])
    if len(stored) == 0:
        peers = tool_rest.get_peers(content)
        if peers.count > 0:
            names = []
            for peer in peers.get():
                if peer == content["symbol"]:
                    continue # skip including the query symbol in peers
                name = get_name_1(dict(q=peer, exchange=content["exchange"], query=content["query"]))
                if name != StopGeneration().result:
                    data = {"symbol": peer, "name": name}
                    names.append(data)
            tool_rag.add_peers_document(content["query"], names, content["symbol"], "get_peers_1", content['grouping'])
            return names
        return StopGeneration().result
    return json.loads(stored[0].docs)["peers"]

def local_datetime(content):
    local_t = []
    for timestamp in content["t"]:
        local_t.append(local_date_from_epoch(timestamp))
    return local_t

def local_date_from_epoch(timestamp):
    if len(str(timestamp)) == 13:
        return datetime.fromtimestamp(timestamp/1000, tz=GeneratedEvent.tz()).strftime('%c')
    else:
        return datetime.fromtimestamp(timestamp, tz=GeneratedEvent.tz()).strftime('%c')

def get_financials_1(content):
    stored = tool_rag.get_basic_financials(content["query"], content["symbol"], "get_financials_1")
    if len(stored) == 0:
        return tool_rest.get_basic_financials(content)
    return [chunk.docs for chunk in stored]

def get_news_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "get_news_1")
    if len(stored) == 0:
        return tool_rest.get_news_simple(content)
    return [NewsTypeFinn.model_validate_json(news.docs).summary().model_dump_json() for news in stored]

def get_daily_candle_2(content):
    stored = tool_rag.get_api_documents(
        query=content["query"], topic=content["stocksTicker"], source="daily_candle_2", 
        meta_opt=[{"from_date": content["date"], "adjusted": content["adjusted"]}])
    if len(stored) == 0:
        candle = tool_rest.get_daily_candle(content)
        # Attempt to recover from choosing a holiday.
        candle_date = parse(content["date"])
        if candle.status is RestStatus.NONE and candle_date.weekday() == 0 or candle_date.weekday() == 4:
            if candle_date.weekday() == 0: # index 0 is monday, index 4 is friday
                content["date"] = candle_date.replace(day=candle_date.day-3).strftime("%Y-%m-%d")
            else:
                content["date"] = candle_date.replace(day=candle_date.day-1).strftime("%Y-%m-%d")
            return get_daily_candle_2(content)
        return candle.model_dump_json()
    return [json.loads(candle.docs) for candle in stored]

def get_custom_candle_2(content):
    stored = tool_rag.get_api_documents(
        query=content["query"], topic=content["stocksTicker"], source="custom_candle_2", 
        meta_opt=[{
            "timespan": content["timespan"],
            "adjusted": content["adjusted"],
            "from": content["from"],
            "to": content["to"]}])
    if len(stored) == 0:
        return tool_rest.get_custom_candle(content)
    return [json.loads(candle.docs) for candle in stored]

def get_overview_2(content):
    stored = tool_rag.get_api_documents(content["query"], content["ticker"], "ticker_overview_2")
    if len(stored) == 0:
        return tool_rest.get_overview(content)
    return json.loads(stored[0].docs)

def get_trends_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "trends_1")
    if len(stored) == 0:
        return tool_rest.get_trends_simple(content)
    return [json.loads(trend.docs) for trend in stored]

def get_news_2(content):
    timestamp_from = parse(content["published_utc.gte"]).timestamp()
    timestamp_to = parse(content["published_utc.lte"]).timestamp()
    news_from = tool_rag.get_api_documents(
        content["query"], content["ticker"], "get_news_2", [{"published_utc": timestamp_from}])
    news_to = tool_rag.get_api_documents(
        content["query"], content["ticker"], "get_news_2", [{"published_utc": timestamp_to}])
    if len(news_from) > 0 and len(news_to) > 0:
        stored = tool_rag.get_api_documents(
            content["query"], content["ticker"], "get_news_2",
            [{"published_utc": {"$gte": timestamp_from}},
             {"published_utc": {"$lte": timestamp_to}}])
        return [NewsTypePoly.model_validate_json(news.docs).summary().model_dump_json() for news in stored]
    return tool_rest.get_news_tagged(content)
        
finance_tool = types.Tool(
    function_declarations=[
        decl_get_symbol_1,
        decl_get_symbols_1,
        decl_get_name_1,
        decl_get_symbol_quote_1,
        decl_get_market_status_1,
        decl_get_market_session_1,
        decl_get_company_peers_1,
        decl_get_local_datetime,
        decl_get_last_market_close,
        decl_get_exchange_codes_1,
        decl_get_exchange_code_1,
        decl_get_financials_1,
        decl_get_daily_candlestick_2,
        decl_get_custom_candlestick_2,
        decl_get_ticker_overview_2,
        decl_get_recommendation_trends_1,
        decl_get_news_with_sentiment_2,
        decl_get_rag_tool_response,
        decl_get_wiki_tool_response,
        decl_get_search_tool_response
    ]
)

function_handler = {
    "get_symbol_1": get_symbol_1,
    "get_symbols_1": get_symbols_1,
    "get_name_1": get_name_1,
    "get_symbol_quote_1": get_quote_1,
    "get_market_status_1": get_market_status_1,
    "get_market_session_1": get_session_1,
    "get_company_peers_1": get_peers_1,
    "get_local_datetime": local_datetime,
    "get_last_market_close": last_market_close,
    "get_exchange_codes_1": get_exchange_codes_1,
    "get_exchange_code_1": get_exchange_code_1,
    "get_financials_1": get_financials_1,
    "get_daily_candlestick_2": get_daily_candle_2,
    "get_custom_candlestick_2": get_custom_candle_2,
    "get_ticker_overview_2": get_overview_2,
    "get_recommendation_trends_1": get_trends_1,
    "get_news_with_sentiment_2": get_news_2,
    "get_rag_tool_response": ask_rag_tool,
    "get_wiki_tool_response": ask_wiki_tool,
    "get_search_tool_response": ask_search_tool
}

## Define the System Prompt

In [24]:
# Define the system prompt.

instruction = f"""You are a helpful and informative bot that answers finance and stock market questions. 
Only answer the question asked and do not change topic. While the answer is still
unknown you must follow these rules for predicting function call order:

RULE#1: Always consult your other functions before get_search_tool_response.
RULE#2: Always consult get_wiki_tool_response before get_search_tool_response.
RULE#3: Always consult get_search_tool_response last.
RULE#4: Always convert timestamps with get_local_datetime and use the converted date/time in your response.
RULE#5: Always incorporate as much useful information from tools and functions in your response."""

## Import the Rest API Keys

In [25]:
# Import the finance api secret keys.

POLYGON_API_KEY = UserSecretsClient().get_secret("POLYGON_API_KEY")
FINNHUB_API_KEY = UserSecretsClient().get_secret("FINNHUB_API_KEY")

## The Function Caller

In [26]:
# Implement the function calling expert.

@retry.Retry(
    predicate=is_retriable,
    initial=2.0,
    maximum=64.0,
    multiplier=2.0,
    timeout=600,
)
def send_message(prompt):
    #display(Markdown("#### Prompt"))
    #print(prompt, "\n")
    memory.set_prompt(prompt)
    # Enable system prompt, function calling and minimum-randomness.
    config_fncall = types.GenerateContentConfig(
        system_instruction=instruction,
        tools=[finance_tool],
        temperature=0.0
    )
    # Handle cases with multiple chained function calls.
    function_calling_in_process = True
    # Send the initial user prompt and function declarations.
    response = api.retriable(api.client.models.generate_content,
                             model=api(Gemini.Model.GEN),
                             config=config_fncall,
                             contents=memory.contents)
    while function_calling_in_process:
        # A part can be a function call or natural language response.
        for part in response.candidates[0].content.parts:
            if function_call := part.function_call:
                # Extract the function call.
                fn_name = function_call.name
                #display(Markdown("#### Predicted function name"))
                #print(fn_name, "\n")
                # Extract the function call arguments.
                fn_args = {key: value for key, value in function_call.args.items()}
                #display(Markdown("#### Predicted function arguments"))
                #print(fn_args, "\n")
                # Call the predicted function.
                api_response = function_handler[fn_name](fn_args)[:20000] # Stay within the input token limit
                #display(Markdown("#### API response"))
                #print(api_response[:500], "...", "\n")
                # Create an API response part.
                api_response_part = types.Part.from_function_response(
                    name=fn_name,
                    response={"content": api_response},
                )
                memory.update_contents(function_call, api_response_part)
                # Send the updated prompt.
                response = api.retriable(api.client.models.generate_content,
                                         model=api(Gemini.Model.GEN),
                                         config=config_fncall,
                                         contents=memory.contents)
            else:
                # Response may be a summary or reasoning step.
                if len(response.candidates[0].content.parts) == 1:
                    function_calling_in_process = False
                    memory.set_summary(response.text.replace("$", "\\$"))
                    break # No more parts in response.
                else:
                    #display(Markdown("#### Natural language reasoning step"))
                    #print(response)
                    memory.set_reason(response.candidates[0].content.parts[0].text)
                    continue # Next part contains a function call.
        if not function_calling_in_process:
            break # The function calling chain is complete.
            
    # Show the final natural language summary.
    display(Markdown("#### Natural language response"))
    display(Markdown(memory.summary))

# Ask a question

<span style="font-size:18px;">
    If you're on free-tier of Gemini you probably want to Run-before here. Your usage tier can be configured in the api-helper at the top of the notebook.
</span>

In [27]:
send_message("What is the current session for US exchanges?")

Upsert chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

The current market session for US exchanges is closed.


In [28]:
send_message("What is the US market status?")

#### Natural language response

The US market is currently closed as of Thu Aug 28 22:52:42 2025 America/New_York time. There is no holiday today. The market session is closed.


In [29]:
send_message("When was the last US market close?")

#### Natural language response

The last market close in the US was on Thu Aug 28 20:00:00 2025, which converts to Fri Aug 29 05:20:00 2025 in local time.


In [30]:
send_message("What is Apple's stock ticker?")

Score similarity to query: 100%|██████████| 10/10 [00:00<00:00, 11.70it/s]
Generate api embedding: 0it [00:00, ?it/s]


#### Natural language response

The stock ticker for Apple is AAPL.


In [31]:
send_message("What is the current price of Amazon stock? Display the result as a json object.")

Generate quote embedding: 0it [00:00, ?it/s]


#### Natural language response

Here is the current price of Amazon (AMZN) stock:
```json
{
  "currentPrice": 231.6,
  "change": 2.48,
  "percentChange": 1.0824,
  "highPriceOfTheDay": 232.71,
  "lowPriceOfTheDay": 228.02,
  "openPriceOfTheDay": 229.005,
  "previousClosePrice": 229.12,
  "timestamp": "Thu Aug 28 16:00:00 2025"
}
```

api.refill_rpm  2000


In [33]:
send_message("""Show me Apple's basic financials and help me understand key performance metrics. 
                How has the stock performed?""")

#### Natural language response

Here's a concise and detailed summary of Apple's (AAPL) basic financials and key performance metrics, including its stock performance:

**Apple (AAPL) Basic Financials and Key Performance Metrics:**

As of the latest available data (primarily Trailing Twelve Months as of June 28, 2025, or September 28, 2024, where specified):

*   **Market Capitalization**: \$3,369,362 million
*   **Price-to-Earnings (Trailing Twelve Months)**: 30.87 (as of June 28, 2025)
*   **Forward Price-to-Earnings**: 31.56
*   **Price-to-Sales (Trailing Twelve Months)**: 7.50 (as of June 28, 2025)
*   **Price-to-Book (Quarterly)**: 46.55 (as of June 28, 2025)
*   **Price-to-Free Cash Flow (Trailing Twelve Months)**: 31.86 (as of June 28, 2025)
*   **Earnings Per Share (Quarterly)**: \$1.57 (as of June 28, 2025)
*   **Free Cash Flow Per Share (Trailing Twelve Months)**: \$6.47 (as of June 28, 2025)
*   **Book Value (Quarterly)**: \$65,830 million (as of June 28, 2025)

**Profitability and Efficiency:**

*   **Net Margin (Trailing Twelve Months)**: 24.92% (as of June 28, 2025)
*   **Operating Margin (Trailing Twelve Months)**: 29.99% (as of June 28, 2025)
*   **Gross Margin (Trailing Twelve Months)**: 46.49% (as of June 28, 2025)
*   **Return on Assets (Trailing Twelve Months)**: 28.95% (as of June 28, 2025)
*   **Return on Equity (Trailing Twelve Months)**: 154.92% (as of June 28, 2025)
*   **Return on Invested Capital (Trailing Twelve Months)**: 60.20% (as of June 28, 2025)
*   **Asset Turnover (Trailing Twelve Months)**: 1.19 (as of June 28, 2025)
*   **Inventory Turnover (Trailing Twelve Months)**: 36.04 (as of June 28, 2025)
*   **Receivables Turnover (Trailing Twelve Months)**: 16.23 (as of June 28, 2025)

**Liquidity and Solvency:**

*   **Current Ratio (Quarterly)**: 0.87 (as of June 28, 2025)
*   **Cash Ratio (Quarterly)**: 0.26 (as of June 28, 2025)
*   **Quick Ratio (Quarterly)**: 0.83 (as of June 28, 2025)
*   **Long-Term Debt to Equity (Quarterly)**: 1.25 (as of June 28, 2025)
*   **Total Debt to Total Equity (Quarterly)**: 1.54 (as of June 28, 2025)

**Stock Performance:**

*   **52-Week High**: \$260.10 (on December 26, 2024)
*   **52-Week Low**: \$169.21 (on April 08, 2025)
*   **52-Week Price Return**: 1.46%
*   **13-Week Price Return**: 15.12%
*   **Year-to-Date Price Return**: -7.96%

Apple's financials indicate a highly profitable company with strong returns on equity and invested capital. Its valuation ratios (Price-to-Earnings, Price-to-Sales, Price-to-Book, Price-to-Free Cash Flow) suggest a premium valuation by the market. While the company has a significant amount of debt relative to its equity, its profitability metrics remain robust. In terms of stock performance, Apple has seen a positive return over the last 13 weeks and 52 weeks, but a negative return year-to-date.

api.refill_rpm  1000


In [34]:
send_message("I need Apple's daily candlestick from 2025-05-05")

Upsert chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

On May 5, 2025, Apple's stock (AAPL) had the following daily candlestick data: the open price was 203.1, the high price of the day was 204.1, the low price of the day was 198.21, and the close price was 198.89. The trading volume for the day was 69,018,452. The pre-market price was 205.0, and the after-hours price was 198.6.


In [35]:
send_message("Tell me who are Apple's peers?")

Score similarity to query: 100%|██████████| 4/4 [00:00<00:00, 17.93it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 5/5 [00:00<00:00, 24.06it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  4.64it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  4.92it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  4.20it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  4.49it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 2/2 [00:00<00:00,  9.35it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 2/2 [00:00<00:00,  9.51it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  4.60it/s]

#### Natural language response

Apple's peers, based on the same subIndustry and country, include Dell Technologies -C (DELL), Hewlett Packard Enterprise (HPE), HP Inc (HPQ), Pure Storage Inc - Class A (PSTG), Super Micro Computer Inc (SMCI), Western Digital Corp (WDC), NetApp Inc (NTAP), IonQ Inc (IONQ), Sandisk Corp (SNDK), Diebold Nixdorf Inc (DBD), and Compusecure Inc-A (CMPO).


api.refill_rpm  2000


In [36]:
send_message("Tell me who are Amazon's peers?")

Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  4.54it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 2/2 [00:00<00:00,  9.50it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  2.25it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 4/4 [00:00<00:00, 19.53it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 2/2 [00:00<00:00,  9.75it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 11/11 [00:00<00:00, 50.35it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  4.59it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  3.93it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  4.51it/

#### Natural language response

Amazon's peers in the subIndustry grouping are: Coupang Inc (CPNG), Ebay Inc (EBAY), Dillards Inc-Cl A (DDS), Ollie's Bargain Outlet Holding (OLLI), Etsy Inc (ETSY), Macy's Inc (M), Savers Value Village Inc (SVV), Kohls Corp (KSS), Groupon Inc (GRPN), and Contextlogic Holdings Inc (LOGC).


In [37]:
send_message("Download the recommendation trends for all of Apple's peers individually then compare them.")

Upsert chunks embedding: 0it [00:00, ?it/s]
Upsert chunks embedding: 0it [00:00, ?it/s]
Upsert chunks embedding: 0it [00:00, ?it/s]
Upsert chunks embedding: 0it [00:00, ?it/s]
Upsert chunks embedding: 0it [00:00, ?it/s]
Upsert chunks embedding: 0it [00:00, ?it/s]
Upsert chunks embedding: 0it [00:00, ?it/s]
Upsert chunks embedding: 0it [00:00, ?it/s]
Upsert chunks embedding: 0it [00:00, ?it/s]
Upsert chunks embedding: 0it [00:00, ?it/s]
Upsert chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

As of August 1, 2025, the recommendation trends for Apple's peers, operating in the same sub-industry, show a generally positive outlook with varying degrees of "Strong Buy," "Buy," "Hold," "Sell," and "Strong Sell" recommendations.

**Companies with the Strongest Positive Sentiment (High "Strong Buy" and "Buy" recommendations):**

*   **DELL Technologies - Class C (DELL):** Leads with a combined 26 "Strong Buy" (7) and "Buy" (19) recommendations.
*   **Western Digital Corporation (WDC):** Shows strong positive sentiment with 25 combined "Strong Buy" (6) and "Buy" (19) recommendations.
*   **Pure Storage Inc - Class A (PSTG):** Has a notable 20 combined "Strong Buy" (8) and "Buy" (12) recommendations.

**Companies with the Highest "Hold" Sentiment:**

*   **NetApp Inc (NTAP):** Has the highest number of "Hold" recommendations at 16.
*   **HP Inc (HPQ):** Also shows a significant number of "Hold" recommendations at 14.
*   **Hewlett Packard Enterprise (HPE):** Has 11 "Hold" recommendations.

**Companies with "Sell" Recommendations:**

*   **Super Micro Computer Inc (SMCI):** Has the highest number of "Sell" recommendations with 3.
*   **HP Inc (HPQ):** Has 1 "Sell" recommendation.
*   **Pure Storage Inc - Class A (PSTG):** Has 1 "Sell" recommendation.

**Companies with No "Sell" or "Strong Sell" Recommendations:**

*   DELL Technologies - Class C (DELL)
*   Hewlett Packard Enterprise (HPE)
*   Western Digital Corporation (WDC)
*   NetApp Inc (NTAP)
*   IONQ Inc (IONQ)
*   SanDisk Corporation (SNDK)
*   Diebold Nixdorf Inc (DBD)
*   CompoSecure Inc - A (CMPO)

In summary, DELL Technologies and Western Digital Corporation exhibit the most favorable analyst recommendations. NetApp Inc, HP Inc, and Hewlett Packard Enterprise show a more mixed sentiment with a substantial portion of "Hold" recommendations. Super Micro Computer Inc is the only peer with a notable number of "Sell" recommendations. The remaining companies generally have fewer total analyst recommendations but still lean towards positive or neutral sentiment.

api.refill_rpm  1000


In [38]:
send_message(
    """Tell me Amazon's current share price and provide candlestick data for the past month.
    Sort the data in descending order by date. Format the prices consistently as currency.
    Round prices to two decimal places.
    Present the data with multiple columns for display in markdown.
    Discuss and provide details about any patterns you notice in the price data.
    Correlate recent patterns with news over the same date range.""")

Score similarity to query: 100%|██████████| 2/2 [00:00<00:00,  4.43it/s]
Generate api embedding: 0it [00:00, ?it/s]
Upsert chunks embedding: 0it [00:00, ?it/s]
Add chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

Amazon's (AMZN) current share price is **\$231.60**.

Here is the candlestick data for Amazon for the past month, sorted in descending order by date:

| Date | Open Price | High Price | Low Price | Close Price | Volume |
|:---|:---|:---|:---|:---|:---|
| Wed Aug 27 2025 | \$228.57 | \$229.87 | \$227.81 | \$229.12 | 21,254,479 |
| Tue Aug 26 2025 | \$227.11 | \$229.00 | \$226.02 | \$228.71 | 26,105,373 |
| Mon Aug 25 2025 | \$227.35 | \$229.60 | \$227.31 | \$227.94 | 22,633,695 |
| Fri Aug 22 2025 | \$222.79 | \$229.14 | \$220.82 | \$228.84 | 37,315,341 |
| Thu Aug 21 2025 | \$222.65 | \$222.78 | \$220.50 | \$221.95 | 32,140,459 |
| Wed Aug 20 2025 | \$227.12 | \$227.27 | \$220.92 | \$223.81 | 36,604,319 |
| Tue Aug 19 2025 | \$230.09 | \$230.53 | \$227.12 | \$228.01 | 29,891,012 |
| Mon Aug 18 2025 | \$230.22 | \$231.91 | \$228.33 | \$231.49 | 25,248,890 |
| Fri Aug 15 2025 | \$232.58 | \$234.08 | \$229.81 | \$231.03 | 39,649,244 |
| Thu Aug 14 2025 | \$227.40 | \$233.11 | \$227.02 | \$230.98 | 61,545,824 |
| Wed Aug 13 2025 | \$222.00 | \$224.92 | \$222.00 | \$224.56 | 36,508,335 |
| Tue Aug 12 2025 | \$222.23 | \$223.50 | \$219.05 | \$221.47 | 37,254,707 |
| Mon Aug 11 2025 | \$221.78 | \$223.05 | \$220.40 | \$221.30 | 31,646,222 |
| Fri Aug 8 2025 | \$223.14 | \$223.80 | \$221.88 | \$222.69 | 32,970,477 |
| Thu Aug 7 2025 | \$221.00 | \$226.22 | \$220.82 | \$223.13 | 40,603,513 |
| Wed Aug 6 2025 | \$214.70 | \$222.65 | \$213.74 | \$222.31 | 54,823,045 |
| Tue Aug 5 2025 | \$213.05 | \$216.30 | \$212.87 | \$213.75 | 51,505,121 |
| Mon Aug 4 2025 | \$217.40 | \$217.44 | \$211.42 | \$211.65 | 77,890,146 |
| Fri Aug 1 2025 | \$217.21 | \$220.44 | \$212.80 | \$214.75 | 122,258,801 |
| Thu Jul 31 2025 | \$235.77 | \$236.53 | \$231.40 | \$234.11 | 104,357,263 |
| Wed Jul 30 2025 | \$231.64 | \$231.80 | \$229.29 | \$230.19 | 32,993,273 |
| Tue Jul 29 2025 | \$234.15 | \$234.72 | \$230.31 | \$231.01 | 33,716,220 |
| Mon Jul 28 2025 | \$233.35 | \$234.29 | \$232.25 | \$232.79 | 26,300,138 |

**Price Patterns and News Correlation:**

**Early August Decline (July 31 - August 1):**
Amazon's stock experienced a notable drop from **\$234.11 on July 31st to \$214.75 on August 1st**, accompanied by exceptionally high trading volume (over 122 million shares). This decline was primarily driven by investor concerns following Amazon's second-quarter earnings report. News articles from August 1st indicated that despite beating revenue and earnings expectations, the stock plummeted due to **slower-than-expected Amazon Web Services (AWS) growth** (18% year-over-year) compared to competitors like Microsoft (39%) and Google Cloud (32%). Additionally, **potential impacts from new tariffs** contributed to investor apprehension. Reports also highlighted increased operating expenses and decreased free cash flow, dampening investor enthusiasm.

**Mid-August Recovery and Stability (August 4 - August 13):**
Following the sharp decline, Amazon's stock entered a period of recovery and relative stability, trading mostly within the **\$211 to \$225 range**. There was a gradual upward trend from August 4th (\$211.65) to August 13th (\$224.56). During this time, news articles generally focused on Amazon's long-term potential and strategic investments. Several reports, such as "Amazon Enters Correction Zone: Time to Panic or Be Brave?" (August 5th), suggested that the initial stock correction was a healthy market reset rather than a fundamental issue. Analysts continued to recommend Amazon as a strong investment, citing its market leadership in e-commerce, cloud computing, and digital advertising, along with its ongoing **Artificial Intelligence (AI) initiatives**. News about Amazon's aggressive expansion in grocery delivery and its rapidly growing advertising business also contributed to a positive long-term outlook.

**Late August Upward Trend (August 15 - August 27):**
Towards the latter half of August, Amazon's stock demonstrated a consistent upward trend, moving from approximately **\$231.03 on August 15th to \$229.12 on August 27th**. This sustained growth was supported by continued positive news regarding Amazon's strategic positioning and AI advancements. Articles like "1 Unstoppable Stock That Could Join Nvidia, Microsoft, and Apple in the \$3 Trillion Club by 2027" (August 15th) and "Where Will Amazon Stock Be in 5 Years?" (August 16th) reiterated the company's significant market valuation potential and strong second-quarter performance. Reports also highlighted Amazon's collaborations and partnerships in AI, such as with Domo, further boosting positive sentiment. The overall positive market sentiment for major technology stocks, with Amazon frequently mentioned as a key player in the "Magnificent Seven," also played a role in its sustained upward trajectory.

In [39]:
send_message("What is Apple's ticker overview")

Upsert chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

Apple Incorporated (ticker: AAPL) is a large company that is traded on the XNAS exchange. It is based in Cupertino, CA. Its CIK is 0000320193, composite FIGI is BBG000B9XRY4, and share class FIGI is BBG001S5N8V8. The market capitalization is 3420561491100.0 United States dollars. Apple's phone number is (408) 996-1010. The company employs 164000 people. Apple's listing date was 1980-12-12. Apple's logo can be found at https://api.polygon.io/v1/reference/company-branding/YXBwbGUuY29t/images/2025-04-04_logo.svg, and its icon can be found at https://api.polygon.io/v1/reference/company-branding/YXBwbGUuY29t/images/2025-04-04_icon.png. The number of share class shares outstanding is 14840390000, and the weighted shares outstanding is 14840390000. The round lot is 100. Apple is among the largest companies in the world, with a broad portfolio of hardware and software products targeted at consumers and businesses. Apple's iPhone makes up a majority of the firm sales, and Apple's other products like Mac, iPad, and Watch are designed around the iPhone as the focal point of an expansive software ecosystem. Apple has progressively worked to add new applications, like streaming video, subscription bundles, and augmented reality. The firm designs its own software and semiconductors while working with subcontractors like Foxconn and TSMC to build its products and chips. Slightly less than half of Apple's sales come directly through its flagship stores, with a majority of sales coming indirectly through partnerships and distribution.


In [40]:
send_message("Tell me about Amazon's historical and current recommendation trends")

Upsert chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

Here is a summary of Amazon's recommendation trends:

As of August 1, 2025, the analyst recommendation trends for Amazon (AMZN) are: 51 buy recommendations, 4 hold recommendations, 0 sell recommendations, 24 strong buy recommendations, and 0 strong sell recommendations.

The recommendation trends for the previous three months are as follows:
*   July 1, 2025: 50 buy, 5 hold, 0 sell, 24 strong buy, 0 strong sell
*   June 1, 2025: 50 buy, 5 hold, 0 sell, 24 strong buy, 0 strong sell
*   May 1, 2025: 51 buy, 6 hold, 0 sell, 22 strong buy, 0 strong sell


api.refill_rpm  2000


In [41]:
send_message("What is Google's stock ticker symbol?")

Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


#### Natural language response

Google's stock ticker symbols on the NASDAQ stock exchange are GOOGL and GOOG. The ticker symbol on the Frankfurt Stock Exchange is GGQ1. These symbols refer to Alphabet Inc., Google's holding company.


In [42]:
send_message("What is MGM Studio's stock symbol?")

Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


#### Natural language response

I am sorry, I was unable to find the stock ticker symbol for Mgm Studios.


api.refill_rpm  2000


In [43]:
send_message("What is MGM Studio's owner company stock symbol?")

Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


#### Natural language response

Amazon owns Mgm Studios. Amazon acquired Mgm in May 2021 for \$8.45 billion, and the studio continues to operate as a label alongside Amazon Studios and Amazon Prime Video. Amazon's stock symbol is AMZN.


In [44]:
send_message("What is Facebook's stock ticker symbol?")

Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


#### Natural language response

The stock ticker symbol for Meta Platforms, Inc., formerly known as Facebook, is META. It is traded on the Nasdaq stock exchange.


api.refill_rpm  2000


In [46]:
send_message('''Compare Amazon's bullish versus bearish predictions from May 01 2025 until today.
                Include a discussion of recommendation trends, and sentiment analysis of news from the same dates.
                Discuss any patterns or correlations you find.''')

api.refill_rpm  1000


#### Natural language response

From May 1, 2025, to August 28, 2025, Amazon (AMZN) has consistently received an overwhelmingly bullish outlook from analysts, with a strong correlation to positive news sentiment.

**Recommendation Trends:**
Analyst recommendations for Amazon remained highly positive throughout this period. In May 2025, there were 22 "Strong Buy" and 51 "Buy" recommendations, with only 6 "Hold" ratings and no "Sell" or "Strong Sell" recommendations. By August 2025, the "Strong Buy" recommendations slightly increased to 24, while "Buy" recommendations remained robust at 51. "Hold" ratings saw a minor decrease to 4, and there continued to be no "Sell" or "Strong Sell" ratings. This consistent trend indicates a firm belief among analysts in Amazon's continued growth and strong market position.

**Sentiment Analysis of News:**
The news sentiment largely mirrored the bullish analyst recommendations, with a strong focus on Amazon's strategic advantages and growth drivers:

**Bullish Predictions and Positive Sentiment:**
*   **Artificial Intelligence (AI) and Cloud Computing (Amazon Web Services - AWS):** This was the most prominent positive theme. Numerous articles highlighted Amazon's substantial investments in AI, its leadership in cloud computing through AWS, and the transformative potential of AI to enhance efficiency, reduce costs, and open new revenue streams across its diverse businesses. AWS was frequently cited as a primary profit engine and a key factor in Amazon's long-term expansion.
*   **E-commerce Dominance and Diversification:** Amazon's strong foothold in the e-commerce market, its expanding Prime membership, and its successful diversification into high-margin sectors like digital advertising and healthcare were consistently praised.
*   **Robotics and Automation:** The company's extensive deployment of robots in fulfillment centers and its advancements in robotics were seen as crucial for boosting productivity and optimizing operational costs.
*   **Strategic Collaborations:** Partnerships in AI, quantum computing, and advertising (e.g., with Roku) were viewed favorably, indicating Amazon's proactive approach to innovation.
*   **Strong Financial Performance:** Many reports noted Amazon's ability to surpass earnings estimates, achieve robust revenue growth, and maintain healthy profit margins.
*   **Investor Confidence:** Billionaire investors like Bill Ackman and Warren Buffett's Berkshire Hathaway either increased their stakes or maintained significant positions in Amazon, further solidifying positive market sentiment. Wall Street analysts frequently issued bullish price targets and "Buy" ratings.

**Bearish Predictions and Neutral Sentiment:**
While the overall sentiment was positive, some articles presented neutral or slightly negative perspectives, often related to specific challenges:
*   **Tariffs and Trade Tensions:** Early in the period (May), concerns were raised about the impact of US sanctions and tariffs on Amazon's business, potentially leading to higher prices and affecting e-commerce sales. However, Amazon's resilience and ability to mitigate these impacts were also noted.
*   **Competition in Cloud and AI:** Although AWS leads the cloud market, some articles highlighted competitors like Microsoft Azure and Google Cloud growing faster in certain segments or developing their own AI chips, posing a competitive challenge.
*   **Slower Growth in Specific Segments:** Occasional reports mentioned a deceleration in online sales or cloud computing growth compared to previous periods, or conservative guidance for upcoming quarters, which sometimes led to temporary stock pullbacks.
*   **Valuation Concerns:** A few articles suggested that Amazon's stock might be trading at a high valuation, implying that its price already factored in significant future growth.
*   **Workforce Reduction:** Amazon Chief Executive Officer Andy Jassy's comments about AI and automation potentially reducing the workforce were noted, though often framed as a strategic move for efficiency rather than a negative indicator.
*   **Project Kuiper Challenges:** Delays and intense competition in Amazon's satellite internet project were identified as potential headwinds.

**Patterns and Correlations:**
A clear pattern emerges: the consistently bullish analyst recommendations are strongly correlated with the predominantly positive news sentiment. The market and analysts appear to prioritize Amazon's long-term growth potential, driven by its technological innovation and market dominance, particularly in AI and cloud computing. Short-term challenges, such as tariff impacts or competitive pressures, were generally perceived as manageable or temporary, often leading to "buy the dip" recommendations rather than a shift to a bearish outlook. This indicates a deep-seated confidence in Amazon's strategic direction and its ability to capitalize on emerging technological trends.

In [52]:
send_message('''Compare Google's bullish versus bearish predictions from May 01 2025 until today.
                Include a discussion of recommendation trends, and sentiment analysis of news from the same dates.
                Discuss any patterns or correlations you find.''')

api.refill_rpm  1000


#### Natural language response

From May 1, 2025, to August 28, 2025, Alphabet Inc. (GOOGL) experienced a predominantly bullish sentiment from both financial analysts and news coverage.

**Recommendation Trends:**
Analyst recommendations for Alphabet Inc. Class A (GOOGL) consistently reflected a positive outlook. "Strong Buy" recommendations saw a slight increase from 17 in May to 18 in June, July, and August. "Buy" recommendations remained stable at 33 in May and June, dipped marginally to 32 in July, but then significantly increased to 37 by August. "Hold" recommendations slightly decreased from 14 in May to 13 in June, July, and August. Notably, there were no "Sell" or "Strong Sell" recommendations throughout this period, indicating a strong and sustained bullish sentiment from analysts.

**Sentiment Analysis of News:**
An analysis of news articles related to Alphabet Inc. (GOOGL) during this period revealed a majority of positive articles (50), a substantial number of neutral articles (50), and a smaller number of negative articles (7).

*   **Bullish (Positive) News:** Many articles highlighted Alphabet's robust performance in artificial intelligence (AI), cloud services (Google Cloud), and core businesses like Search and YouTube. Key positive themes included exceeding earnings expectations, increased institutional investment, analyst price target upgrades, significant advancements in AI (Gemini models, AI-powered search, robotics platforms), and strategic partnerships (e.g., with Nokia, XMPro, ATX Defense, Odoo, SMX, OpenAI, MiLi, Brookfield Asset Management, and Uber's Waymo). Strong revenue growth, attractive valuations, and share repurchase programs were also frequently cited as positive indicators. Alphabet was also positioned as a leader in quantum computing and a promising long-term investment.
*   **Bearish (Negative) News:** Negative sentiment was primarily driven by concerns related to antitrust rulings (e.g., potential impact on Apple's profits, Epic Games lawsuit), perceived threats to Google's core search engine business from emerging large language models, and the environmental impact of water usage by data centers. Some articles also reported stock price drops due to specific news or broader market concerns.
*   **Neutral News:** A significant portion of the news was neutral, often discussing upcoming earnings reports, general market volatility, comparisons with competitors without a strong directional sentiment, routine corporate actions (e.g., app delistings), or partnerships where Google's role was more as a platform provider rather than a direct performance indicator. Some articles also presented mixed outlooks, acknowledging both challenges and opportunities.

**Patterns and Correlations:**
There is a strong correlation between the consistently bullish analyst recommendations and the predominantly positive sentiment observed in news articles. The high number of "Strong Buy" and "Buy" ratings from analysts aligns with the numerous news pieces highlighting Alphabet's strong financial performance, leadership in AI, and strategic business expansions.

The negative news, while present, did not significantly deter the overall bullish sentiment from analysts. This suggests that the market and analysts largely viewed these negative developments (such as antitrust concerns and AI competition to search) as manageable risks or as opportunities for Alphabet to adapt and innovate, rather than fundamental threats to its long-term prospects. For example, while some articles discussed the potential for AI to disrupt Google Search, others emphasized Alphabet's proactive integration of AI into its search and cloud services as a strategic strength. The consistent focus on Alphabet's AI advancements, cloud growth, and diversified business model in positive news articles likely reinforced the bullish analyst sentiment throughout this period.

In [61]:
send_message(
    '''How is the outlook for Apple based on trends and news sentiment from May 01 2025 until today?
    Perform the same analysis on all peers by sub-industry. Then compare Apple result to it's peers.''')

Add chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

Here's a summary of the outlook for Apple and its peers based on recommendation trends and news sentiment from May 1, 2025, to August 29, 2025.

**Apple (AAPL) Analysis**

*   **Recommendation Trends:** Analyst recommendations for Apple in August 2025 show a consensus of "Buy" with 24 buys, 15 holds, and 3 sells. Similar trends were observed in May, June, and July 2025.
*   **News Sentiment:** The news sentiment for Apple is mixed. Recent articles mention both positive and negative aspects.

    *   **Positive:** Apple is recognized for its AI potential, inclusion in growth ETFs, and strategic partnerships. For example, it is part of a new streaming bundle with Rogers Xfinity, is a top holding in AI-driven ETFs, and has a long-term agreement with MP Materials for US-produced magnets.
    *   **Neutral:** Some articles mention Apple in the context of market trends or as part of a bundled service without strong positive or negative sentiment.
    *   **Negative:** Apple is perceived as lagging in AI developments compared to other tech giants. Warren Buffett's Berkshire Hathaway has been reducing its Apple stake. There are also class action lawsuits alleging false statements about AI-based Siri features.

**Peer Analysis**

Apple's peers by sub-industry include: Dell Technologies, Hewlett Packard Enterprise, HP Inc, Pure Storage Inc, Super Micro Computer Inc, Western Digital Corp, NetApp Inc, IonQ Inc, Sandisk Corp, Diebold Nixdorf Inc, and Composecure Inc.

**Comparison to Peers**

*   **Nvidia (NVDA):** Nvidia is frequently mentioned as a leader in AI, with strong revenue growth and market dominance. It is often compared favorably to Apple in terms of AI innovation.
*   **Microsoft (MSFT):** Microsoft is also recognized as a leader in AI and cloud computing, often outperforming Apple in AI developments.
*   **Alphabet (GOOG/GOOGL):** Alphabet is seen as a strong competitor in AI, with potential for significant growth. Some articles suggest it could surpass Apple in market capitalization.

**Summary**

The outlook for Apple is cautiously optimistic. While the company faces challenges in AI and trade, it maintains a strong financial position and brand. Analyst recommendations remain positive, but news sentiment is mixed. Compared to its peers, particularly Nvidia, Microsoft, and Alphabet, Apple is perceived as lagging in AI innovation.


In [55]:
send_message(
    '''What does the recent news say about Apple and the impact of tariffs? From 2025-03-01 up to today.
    Also locate candlestick data for the same dates. 
    Discuss in detail any correlations in patterns between the candlestick and news data.
    Ignore duplicate news entry.''')

Add chunks embedding: 0it [00:00, ?it/s]


api.refill_rpm  2000


Upsert chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

Here's a summary of the news about Apple and the impact of tariffs, along with candlestick data analysis.

**News Summary for Apple (March 1, 2025 - August 28, 2025):**

The news articles present a mixed view of Apple during this period. Here's a breakdown of key themes:

*   **AI Development:** Apple is seen as lagging behind competitors in AI development, but is actively exploring partnerships and acquisitions to enhance its AI capabilities, particularly for Siri. There are reports of Apple considering using Google's Gemini AI.
*   **Tariffs and Trade:** Apple faces potential negative impacts from tariffs, particularly on iPhones manufactured in China. However, the company is also taking steps to mitigate these risks by diversifying production to India and potentially securing tariff exemptions through increased U.S. investments.
*   **Stock Performance and Valuation:** Apple's stock performance has been mixed, with some analysts suggesting it is overvalued and others seeing potential for growth. Warren Buffett's Berkshire Hathaway has reduced its stake in Apple, citing valuation concerns. There are also reports of class action lawsuits against Apple alleging misrepresentations about AI features.
*   **New Products and Services:** Apple is planning to launch innovative AI-powered smart home products by 2027, including a tabletop robot and smart speaker. The company is also working on a foldable iPhone with potential AI tools.
*   **Financial Performance:** Apple continues to demonstrate strong financial performance, paying quarterly dividends and aggressively repurchasing shares.
*   **Partnerships:** Apple is partnering with Rogers in Canada to bundle Apple TV+ with other streaming services. It also has a long-term agreement with MP Materials for U.S.-produced magnets.
*   **Legal Issues:** Apple faces a class action lawsuit alleging false statements about AI-based Siri features for the iPhone 16. Epic Games secured a partial legal win against Apple in an Australian court.

**Candlestick Data Analysis:**

I have retrieved the candlestick data for Apple (AAPL) from March 1, 2025, to August 28, 2025. Due to the limitations of the tool, I am unable to perform a detailed technical analysis or identify specific candlestick patterns. However, I can provide some general observations:

*   The candlestick data shows the daily open, high, low, and close prices for Apple stock during the specified period.
*   The volume data indicates the number of shares traded each day.
*   The candlestick patterns and volume data could be used to identify potential trends, support and resistance levels, and other technical indicators.

**Correlations between Candlestick and News Data:**

Without the ability to perform a more in-depth analysis, it's difficult to pinpoint specific correlations. However, here's how one might look for connections:

*   **News Events and Price Movements:** Significant news events, such as announcements about tariffs, AI partnerships, or product launches, could be correlated with price movements in the candlestick data. For example, a positive news announcement might be followed by a price increase, while a negative announcement could lead to a price decrease.
*   **Volume and Sentiment:** High trading volume could indicate strong market sentiment, either positive or negative, related to a particular news event.
*   **Candlestick Patterns and Trend Changes:** Specific candlestick patterns, such as bullish or bearish engulfing patterns, could signal potential trend changes in response to news events.

**Example:**

If there was news about potential tariffs being imposed on Apple products, you might look for a corresponding drop in the stock price and an increase in trading volume in the candlestick data. Conversely, if there was news about a successful AI partnership, you might look for a price increase and high volume.

**In conclusion:** The recent news about Apple is a mix of positive and negative developments. The company faces challenges related to AI development, tariffs, and competition, but it also has opportunities for growth through new products, services, and strategic partnerships. A thorough analysis of the candlestick data, in conjunction with the news events, could provide valuable insights into the stock's potential future performance.


# Conclusion

<span style="font-size:18px;">
For now that will have to do. Our Essy has a solid foundation but more could be done to organise metadata. No evaluation or validation has been performed (except fuzzing the prompt). Next steps include restructuring the vector database based on lessons learned. That'll be followed by plotting, multi-modal, and structured output. The last close date (generative) function can be temperamental. In the same way Gemini always feels regarding dates. I've learnt so much. I'm happy I decided to participate in the event! It really has been a joy to see Essy grow from random chat with Gemini into the foundation for a good-broker buddy. I hope you enjoy playing with this edition as much as I enjoyed building it!
</span>

# Update June 7, 2025

<span style="font-size:18px;">
    Bugfix version 102 finally brings Essy to a stable milestone. A month and a half late :) There's still more to be built including adding reasoning, agents, and structured output. A few unimplemented rest endpoints remain that could make Essy more self-reliant. The vector store has gotten bigger but not smarter. Essy can tell us pre-scored news has some sentiment but cannot generate it due to limited summaries. Essy can detect interesting patterns in a dataset but not between adjacent datasets. There's so much data we'll need to recruit Essy some help.
</span>

# Advanced (localhost required)

<span style="font-size:18px;">
    The functions demonstrated here require a locally running notebook. A dedicated GPU with at least 8GB VRAM is recommended but not required. Output is generated with Gemma 3 12B QAT, Gemma.cpp, and (later) Gemma 3n. Output on Kaggle is based on cached data.
</span>

In [ ]:
# soon